<a href="https://colab.research.google.com/github/mahb97/joyce-dubliners-similes-analysis/blob/main/02_linguistic_analysis_and_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Joyce Simile Research: Comprehensive Linguistic Analysis and Comparison Framework

# Abstract

This notebook presents a reproducible computational framework for analysing similes in James Joyce’s *Dubliners* and benchmarking extraction methods against a British National Corpus (BNC) baseline. Four datasets are integrated—manual close-reading (ground truth), a restrictive rule-based extractor, a less-restrictive NLP extractor, and BNC—under a harmonised taxonomy that merges *Joycean_Quasi* with the BNC’s *Quasi_Similes* into a single label **Quasi_Similes** to avoid double-counting. The pipeline performs sentence-level linguistic profiling (comparator-span detection, pre/post structure, POS distributions, syntactic complexity, exploratory sentiment) and evaluates extractors via **instance-aligned F1** (exact match then fuzzy ≥ 0.92) against the manual set.

Statistical inference combines a 4-way χ² with standardized residuals and BH-FDR control, two-proportion tests (Newcombe CIs, Cohen’s *h*), binomial checks against BNC reference proportions, and continuous-feature comparisons (Welch *t*, Mann–Whitney *U*, Hedges’ *g*, Cliff’s δ). Topic modelling (LDA) summarises thematic variation per subset.

Results show a strong categorical association across corpora (**χ² ≈ 281.88**, **df = 15**, **Cramér’s V ≈ 0.318**, *p* ≪ .001). **Quasi\_Similes** are more prevalent in **BNC** (≈ 41 %) than in **Joyce-Manual** (≈ 29 %), while Joyce exhibits enrichments in **Joycean\_Framed**, **Joycean\_Quasi\_Fuzzy**, and **Joycean\_Silent** (≈ 20 % combined in the manual set). The less-restrictive NLP subset collapses to **Standard** (control), confirming distributional contrasts are not driven by it. Instance-aligned performance indicates partial recovery of expert labels (Rule-Based vs Manual: micro-F1 ≈ **0.343**, macro-F1 ≈ **0.178**; NLP vs Manual: micro-F1 ≈ **0.292**, macro-F1 ≈ **0.059**), motivating targeted pattern and dependency enhancements.




# 6. Comprehensive Linguistic Analysis Framework

## 6.1 Multi-Dataset Integration
The pipeline ingests four sources and standardises them to a common schema with stable IDs and explicit provenance:
- **Manual_CloseReading** (expert annotations; ground truth)
- **Restrictive_Dubliners** (rule-based extractor)
- **NLP_LessRestrictive_PG** (broad pattern extractor; control—predominantly *Standard*)
- **BNC_Baseline** (standard-English reference)

Column names are normalised, missing IDs are repaired, and categories are **harmonised** so that *Joycean_Quasi* and the BNC’s *Quasi_Similes* are merged into a single label **Quasi_Similes**. This avoids double-counting and supports fair cross-corpus comparisons. All intermediate artefacts (CSV/JSON) are written with timestamps for reproducibility.

## 6.2 Advanced Linguistic Feature Extraction
Using spaCy (with a robust fallback), the framework derives sentence-level linguistic features tailored to simile analysis:
- **Comparator span detection** for `like`, `as`, **as if/as though**, **as … as**, and lemma families (**seem\***, **resembl\***), plus punctuation-mediated comparators (**colon, semicolon, ellipsis, dash**).
- **Pre/Post comparator structure**: token counts on each side and **Pre_Post_Ratio**.
- **Syntactic complexity** via dependency depth; **POS distribution** and **lemmatised text**.
- **Figurative density** from comparator/marker hits.
- **Exploratory sentiment** (TextBlob polarity/subjectivity).
If spaCy is unavailable, the fallback computes token counts, comparator positions, and sentiment only.

## 6.3 Performance Validation
Extractor performance is evaluated **instance-by-instance** against the manual set using **sentence alignment**:
- Exact match first, then **fuzzy matching (≥ 0.92)** to pair sentences.
- Per-category **TP/FP/FN** and **micro/macro F1** are computed from the aligned pairs—more faithful than bag-of-counts.
- Statistical testing (reported in Section 8): **4-way χ²** with standardised residuals and BH-FDR; **two-proportion tests** (Newcombe CIs, Cohen’s *h*), **binomial checks** vs BNC, and **continuous** comparisons (Welch *t*, Mann–Whitney *U*, Hedges’ *g*, Cliff’s δ).  
These procedures emphasise **effect sizes and error control**, and remain informative even when the less-restrictive NLP set collapses to *Standard*.



In [7]:
# =============================================================================
# COMPREHENSIVE LINGUISTIC COMPARISON OF FOUR SIMILE DATASETS
# (UPDATED: multi-comparator pre / between / post segmentation in Cell 1)
# =============================================================================

import os
import re
import json
import warnings
from collections import Counter
from difflib import SequenceMatcher
from datetime import datetime
import random
import sys

import numpy as np
import pandas as pd

# Plot libs not used in this cell but kept for notebook continuity
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

# Optional NLP libs
try:
    import spacy
except Exception:
    spacy = None

try:
    from textblob import TextBlob
except Exception:
    TextBlob = None

print("COMPREHENSIVE LINGUISTIC COMPARISON OF FOUR SIMILE DATASETS (FIXED + SEGMENTS)")
print("=" * 75)
print("Dataset 1: Manual Annotations (Ground Truth - Close Reading)")
print("Dataset 2: Rule-Based Extraction (Restrictive - Domain-Informed)")
print("Dataset 3: NLP Extraction (Less-Restrictive - PG Dubliners)")
print("Dataset 4: BNC Baseline Corpus (Standard English Reference)")
print("=" * 75)

# Initialize spaCy if available
nlp = None
if spacy is not None:
    try:
        nlp = spacy.load("en_core_web_sm")
        print("spaCy pipeline loaded: en_core_web_sm")
    except OSError:
        print("spaCy model not found; attempting to download…")
        os.system("python -m spacy download en_core_web_sm")
        try:
            nlp = spacy.load("en_core_web_sm")
            print("spaCy pipeline loaded after download: en_core_web_sm")
        except Exception:
            print("spaCy unavailable; analysis will use simplified methods.")

class ComprehensiveLinguisticComparator:
    """
    Full pipeline:
      - robust loading & standardisation
      - linguistic feature extraction (spaCy/TextBlob, simplified fallback)
      - multi-comparator segmentation (Pre / Between / Post) + shares & flags
      - category harmonisation (MERGES Joycean_Quasi into Quasi_Similes)
      - instance-aligned F1 (exact + fuzzy sentence matching)
      - reproducibility & environment stamping
      - combined CSV export with stable ordering
    """

    # ----------------- Constructor & environment -----------------
    def __init__(self):
        self.nlp = nlp
        self.datasets = {}
        self.linguistic_features = {}
        self.comparison_results = {}

        # Reproducibility
        self.random_seed = 42
        random.seed(self.random_seed)
        np.random.seed(self.random_seed)

        # Environment info for auditability
        tb_ver = "n/a"
        try:
            import textblob as _tb
            tb_ver = getattr(_tb, "__version__", "n/a")
        except Exception:
            pass
        self.env_info = {
            "python": sys.version,
            "pandas": pd.__version__,
            "numpy": np.__version__,
            "spacy": getattr(spacy, "__version__", "n/a") if spacy is not None else "n/a",
            "textblob": tb_ver,
        }
        print("Environment:", self.env_info)

    # ----------------- ID / Loading / Standardisation -----------------
    def _ensure_ids(self, df, dataset_name, prefix=None):
        """
        Ensure a unique, non-null 'Instance_ID' string column exists.
        If missing, non-unique, or contains NaNs, regenerate sequential IDs with a readable prefix.
        """
        if df is None or df.empty:
            return pd.DataFrame(columns=['Instance_ID'])

        short = (prefix or {
            'manual': 'MAN',
            'rule_based': 'RST',
            'nlp': 'NLP',
            'bnc': 'BNC'
        }.get(dataset_name, dataset_name[:3].upper()))

        candidates = ['Instance_ID', 'ID', 'id', 'sentence_id', 'Sentence_ID', 'Index', 'index']
        chosen = next((c for c in candidates if c in df.columns), None)
        if chosen and chosen != 'Instance_ID':
            df = df.rename(columns={chosen: 'Instance_ID'})
        elif not chosen:
            df['Instance_ID'] = np.nan

        # Normalizes and tests uniqueness
        df['Instance_ID'] = df['Instance_ID'].astype(str).replace({'nan': np.nan, '': np.nan})
        needs_regen = df['Instance_ID'].isna().any() or (not df['Instance_ID'].is_unique)
        if needs_regen:
            df['Instance_ID'] = [f"{short}_{i+1:05d}" for i in range(len(df))]

        return df

    def _load_manual_dataset_robust(self, file_content):
        """Robust loader for manual annotations with long quoted Joycean sentences."""
        import csv
        try:
            df = pd.read_csv(
                file_content, encoding='cp1252', quotechar='"',
                quoting=csv.QUOTE_MINIMAL, skipinitialspace=True, engine='python'
            )
            if 'Sentence Context' in df.columns:
                df = df[df['Sentence Context'].astype(str).str.lower() != 'sentence context'].copy()
                return df
        except Exception as e:
            print(f"  pandas (python engine) failed: {e}")

        # Fallback simpler read
        try:
            df = pd.read_csv(file_content, encoding='cp1252')
            if 'Sentence Context' in df.columns:
                df = df[df['Sentence Context'].astype(str).str.lower() != 'sentence context'].copy()
                return df
        except Exception as e:
            print(f"  pandas (default) failed: {e}")

        print("  Manual annotations not found or failed to load.")
        return pd.DataFrame()

    def load_datasets(self, manual_file=None, rule_based_file=None, nlp_file=None, bnc_file=None):
        print("\nLOADING DATASETS WITH FIXED ID HANDLING & EXPLICIT LABELS")
        print("-" * 70)

        # Manual (close reading)
        print("Loading manual annotations…")
        self.datasets['manual'] = self._load_manual_dataset_robust(manual_file) if manual_file else pd.DataFrame()
        self.datasets['manual'] = self._ensure_ids(self.datasets['manual'], 'manual', prefix='MAN')
        if not self.datasets['manual'].empty:
            self.datasets['manual']['Original_Dataset'] = 'Manual_CloseReading'

        # Rule-based (restrictive)
        print("Loading rule-based (restrictive)…")
        self.datasets['rule_based'] = pd.read_csv(rule_based_file) if rule_based_file else pd.DataFrame()
        self.datasets['rule_based'] = self._ensure_ids(self.datasets['rule_based'], 'rule_based', prefix='RST')
        if not self.datasets['rule_based'].empty:
            self.datasets['rule_based']['Original_Dataset'] = 'Restrictive_Dubliners'

        # NLP (less-restrictive PG)
        print("Loading NLP (less-restrictive PG)…")
        self.datasets['nlp'] = pd.read_csv(nlp_file) if nlp_file else pd.DataFrame()
        self.datasets['nlp'] = self._ensure_ids(self.datasets['nlp'], 'nlp', prefix='NLP')
        if not self.datasets['nlp'].empty:
            self.datasets['nlp']['Original_Dataset'] = 'NLP_LessRestrictive_PG'

        # BNC
        print("Loading BNC baseline…")
        self.datasets['bnc'] = pd.read_csv(bnc_file, encoding='utf-8') if bnc_file else pd.DataFrame()
        self.datasets['bnc'] = self._ensure_ids(self.datasets['bnc'], 'bnc', prefix='BNC')
        if not self.datasets['bnc'].empty:
            self.datasets['bnc']['Original_Dataset'] = 'BNC_Baseline'

        self._standardize_datasets()
        self._standardize_categories()

        for name, df in self.datasets.items():
            print(f"{name:>12}: rows={len(df):4d}  "
                  f"missing_IDs={df['Instance_ID'].isna().sum() if 'Instance_ID' in df else 'N/A'}  "
                  f"missing_Original_Dataset={df['Original_Dataset'].isna().sum() if 'Original_Dataset' in df else 'N/A'}")
        print(f"Total instances: {sum(len(df) for df in self.datasets.values())}")

    def _standardize_datasets(self):
        print("Standardizing column names & adding Dataset_Source…")

        # Manual
        df = self.datasets.get('manual', pd.DataFrame())
        if not df.empty:
            ren = {
                'Category (Framwrok)': 'Category_Framework',
                'Comparator Type ': 'Comparator_Type',
                'Sentence Context': 'Sentence_Context',
                'Page No.': 'Page_Number'
            }
            df = df.rename(columns={k: v for k, v in ren.items() if k in df.columns})
            df['Dataset_Source'] = 'Manual_Expert_Annotation'
            if 'Category_Framework' in df.columns:
                df['Category_Framework'] = df['Category_Framework'].astype(str)
            self.datasets['manual'] = df
        else:
            self.datasets['manual'] = pd.DataFrame(columns=[
                'Instance_ID','Category_Framework','Comparator_Type','Sentence_Context','Page_Number',
                'Dataset_Source','Original_Dataset'
            ])

        # Rule-based
        df = self.datasets.get('rule_based', pd.DataFrame())
        if not df.empty:
            df = df.rename(columns={
                'Sentence Context': 'Sentence_Context',
                'Comparator Type ': 'Comparator_Type',
                'Category (Framwrok)': 'Category_Framework'
            })
            df['Dataset_Source'] = 'Rule_Based_Domain_Informed'
            if 'Category_Framework' in df.columns:
                df['Category_Framework'] = df['Category_Framework'].astype(str)
            self.datasets['rule_based'] = df
        else:
            self.datasets['rule_based'] = pd.DataFrame(columns=[
                'Instance_ID','Category_Framework','Comparator_Type','Sentence_Context',
                'Dataset_Source','Original_Dataset'
            ])

        # NLP (less-restrictive)
        df = self.datasets.get('nlp', pd.DataFrame())
        if not df.empty:
            if 'Sentence_Context' not in df.columns:
                for c in ['Sentence Context','text','sentence','context','content']:
                    if c in df.columns:
                        df = df.rename(columns={c: 'Sentence_Context'})
                        break
            if 'Comparator Type ' in df.columns:
                df = df.rename(columns={'Comparator Type ': 'Comparator_Type'})
            if 'Category (Framwrok)' in df.columns and 'Category_Framework' not in df.columns:
                df = df.rename(columns={'Category (Framwrok)': 'Category_Framework'})
            if 'Category_Framework' not in df.columns:
                df['Category_Framework'] = 'NLP_Basic_Pattern'
            df['Dataset_Source'] = 'NLP_General_Pattern_Recognition'
            df['Category_Framework'] = df['Category_Framework'].astype(str)
            self.datasets['nlp'] = df
        else:
            self.datasets['nlp'] = pd.DataFrame(columns=[
                'Instance_ID','Category_Framework','Comparator_Type','Sentence_Context',
                'Dataset_Source','Original_Dataset'
            ])

        # BNC
        df = self.datasets.get('bnc', pd.DataFrame())
        if not df.empty:
            if 'Category (Framework)' in df.columns and 'Category_Framework' not in df.columns:
                df = df.rename(columns={'Category (Framework)':'Category_Framework'})
            if 'Comparator Type' in df.columns and 'Comparator_Type' not in df.columns:
                df = df.rename(columns={'Comparator Type':'Comparator_Type'})
            if 'Sentence Context' in df.columns and 'Sentence_Context' not in df.columns:
                df = df.rename(columns={'Sentence Context':'Sentence_Context'})
            df['Dataset_Source'] = 'BNC_Standard_English_Baseline'
            if 'Category_Framework' in df.columns:
                df['Category_Framework'] = df['Category_Framework'].astype(str)
            self.datasets['bnc'] = df
        else:
            self.datasets['bnc'] = pd.DataFrame(columns=[
                'Instance_ID','Sentence_Context','Comparator_Type','Category_Framework',
                'Dataset_Source','Original_Dataset'
            ])

        print("Standardization complete.")

    def _standardize_categories(self):
        """
        Harmonize Category_Framework labels.
        IMPORTANT: Merge Joycean_Quasi and its variants into Quasi_Similes (unified quasi-simile phenomenon).
        """
        print("Harmonizing Category_Framework labels…")
        mapping = {
            # Standard variants
            'NLP_Basic': 'Standard',
            'NLP_Basic_Pattern': 'Standard',
            'Standard_English_Usage': 'Standard',
            'Standard': 'Standard',

            # Joycean subtypes
            'Joycean_Framed': 'Joycean_Framed',
            'Joycean_Silent': 'Joycean_Silent',
            'Joycean_Quasi_Fuzzy': 'Joycean_Quasi_Fuzzy',
            'Joycean-Quasi-Fuzzy': 'Joycean_Quasi_Fuzzy',

            # >>> UNIFY all quasi-simile tags here <<<
            'Quasi_Similes': 'Quasi_Similes',
            'Quasi_Simile': 'Quasi_Similes',     # singular → plural canonical
            'Joycean_Quasi': 'Quasi_Similes',    # merge with BNC tag
            'Joycean-Quasi': 'Quasi_Similes',    # hyphen variant

            # mislabels leaking from dataset names → map to Standard
            'NLP_LessRestrictive': 'Standard',
            'NLP_General_Pattern': 'Standard',
            'Less-Restrictive': 'Standard',

            # housekeeping
            'Uncategorised': 'Uncategorized',
            'nan': 'Uncategorized', 'NaN': 'Uncategorized', '': 'Uncategorized'
        }
        for name, df in self.datasets.items():
            if df.empty or 'Category_Framework' not in df.columns:
                continue
            df['Category_Framework'] = df['Category_Framework'].astype(str).map(mapping).fillna(df['Category_Framework'])
            self.datasets[name] = df
        print("Category harmonization complete.")

    # ----------------- Utilities & segmentation helpers -----------------
    _SUBORDINATORS = {"that","who","which","when","because","though","although","if","as",
                      "while","since","unless","until","where","whereas"}

    def _fmt_between_list(self, lst):
        """Return '0' for empty; else 'a, b, c' with no brackets."""
        try:
            if not lst:
                return "0"
            return ", ".join(str(int(x)) for x in lst)
        except Exception:
            return "0"

    @staticmethod
    def _normalize_sentence_for_match(s: str) -> str:
        s = (s or "")
        s = s.replace("—", "-").replace("–", "-")
        s = re.sub(r"\s+", " ", s).strip().lower()
        table = str.maketrans("", "", "\"'“”‘’")
        s = s.translate(table)
        return s

    def _fuzzy_equal(self, a: str, b: str, threshold: float = 0.92) -> bool:
        if not a or not b:
            return False
        ra = self._normalize_sentence_for_match(a)
        rb = self._normalize_sentence_for_match(b)
        if ra == rb:
            return True
        return SequenceMatcher(None, ra, rb).ratio() >= threshold

    def _simple_tokens(self, s: str):
        s = re.sub(r"[“”‘’\"']", "", str(s))
        s = s.replace("—","-").replace("–","-")
        return re.findall(r"\w+|[:;,\.\-\(\)…]", s)

    def _detect_comparator_spans(self, doc_tokens, use_spacy=False):
        """
        Return list of comparator spans as (start_i, end_i, label).
        - Single-token spans use start=end
        - 'as … as' pairs become a single span covering both anchors: label='as_as'
        """
        spans = []
        n = len(doc_tokens)

        def t_text(i): return (doc_tokens[i].text if use_spacy else doc_tokens[i])
        def t_lemma(i):
            if use_spacy:
                return getattr(doc_tokens[i], "lemma_", t_text(i)).lower()
            return t_text(i).lower()

        lower = [t_text(i).lower() for i in range(n)]
        lemmas = [t_lemma(i) for i in range(n)]
        used = set()

        # (1) multiword: 'as if' / 'as though'
        i = 0
        while i < n-1:
            if lower[i] == "as" and lower[i+1] == "if":
                spans.append((i, i+1, "as_if")); used.update((i,i+1)); i += 2; continue
            if lower[i] == "as" and lower[i+1] == "though":
                spans.append((i, i+1, "as_though")); used.update((i,i+1)); i += 2; continue
            i += 1

        # (2) 'as … as'
        as_positions = [i for i,w in enumerate(lower) if w == "as" and i not in used]
        consumed_as = set()
        for i in as_positions:
            if i in consumed_as:
                continue
            partner = None
            for j in as_positions:
                if j > i and (j - i) <= 8 and j not in consumed_as:
                    partner = j; break
            if partner is not None:
                spans.append((i, partner, "as_as"))
                consumed_as.add(i); consumed_as.add(partner)
        used.update(consumed_as)

        # (3) single 'like'
        for i,w in enumerate(lower):
            if w == "like":
                spans.append((i,i,"like"))

        # (4) lemma families: resemble / seem / appear
        for i,lm in enumerate(lemmas):
            if lm.startswith("resembl") or lm.startswith("seem") or lm.startswith("appear"):
                spans.append((i,i,lm.split("-")[0] if "-" in lm else lm))

        # (5) punctuation frames
        for i,w in enumerate([t_text(k) for k in range(n)]):
            if w in {":",";","—","–","-","...","…"}:
                spans.append((i,i,f"punc_{w}"))

        # sort & coalesce overlaps
        spans = sorted(spans, key=lambda x: (x[0], x[1]))
        coalesced, last = [], None
        for s in spans:
            if last is None:
                last = s
            else:
                if s[0] <= last[1] and s[1] >= last[0]:
                    last = (min(last[0], s[0]), max(last[1], s[1]), last[2])
                else:
                    coalesced.append(last); last = s
        if last is not None:
            coalesced.append(last)
        return coalesced

    def _segments_from_spans(self, doc_tokens, spans, use_spacy=False):
        """
        Compute token counts for pre / between / post w.r.t. comparator spans.
        Excludes spaces/punct from counts. Returns dict with counts + flags.
        """
        # Build list of content-token indices
        content_idx = []
        doc_to_content = {}
        for i, tok in enumerate(doc_tokens):
            if use_spacy:
                if getattr(tok, "is_space", False) or getattr(tok, "is_punct", False):
                    continue
            else:
                if re.fullmatch(r"[:;,.\-\(\)…]", str(tok)):
                    continue
            doc_to_content[i] = len(content_idx)
            content_idx.append(i)

        def nearest_kept(i):
            if i in doc_to_content:
                return doc_to_content[i]
            L, R = i-1, i+1
            while L >= 0 or R < len(doc_tokens):
                if L >= 0 and L in doc_to_content: return doc_to_content[L]
                if R < len(doc_tokens) and R in doc_to_content: return doc_to_content[R]
                L -= 1; R += 1
            return 0

        spans_np = []
        for a,b,_label in spans:
            a_np = nearest_kept(a); b_np = nearest_kept(b)
            spans_np.append((min(a_np,b_np), max(a_np,b_np)))
        spans_np = sorted(spans_np)

        total = len(content_idx)
        if total == 0:
            return {
                "Pre_Tokens": 0, "Post_Tokens": 0, "Between_Tokens_List": [],
                "Between_Tokens_Total": 0, "Between_Segments": 0,
                "Between_Max": 0, "Between_Mean": 0.0,
                "Between_Has_Verb": False, "Between_Has_Subordinator": False
            }

        if not spans_np:
            # No comparator detected → neutral split
            pre = total // 2; post = total - pre
            return {
                "Pre_Tokens": pre, "Post_Tokens": post, "Between_Tokens_List": [],
                "Between_Tokens_Total": 0, "Between_Segments": 0,
                "Between_Max": 0, "Between_Mean": 0.0,
                "Between_Has_Verb": False, "Between_Has_Subordinator": False
            }

        first_start = spans_np[0][0]
        last_end   = spans_np[-1][1]

        pre = first_start
        post = max(total - (last_end + 1), 0)

        between = []
        for i in range(len(spans_np)-1):
            gap = max(spans_np[i+1][0] - (spans_np[i][1] + 1), 0)
            between.append(gap)

        # clause-like diagnostics in raw token space
        has_verb = False
        has_sub  = False
        if len(spans) >= 2:
            for i in range(len(spans)-1):
                a_end, b_start = spans[i][1], spans[i+1][0]
                chunk = doc_tokens[a_end+1:b_start]
                for tok in chunk:
                    if use_spacy:
                        if getattr(tok, "pos_", "") == "VERB":
                            has_verb = True
                        if str(getattr(tok, "text", tok)).lower() in self._SUBORDINATORS:
                            has_sub = True
                    else:
                        st = str(tok).lower()
                        if st in self._SUBORDINATORS:
                            has_sub = True
                        if re.fullmatch(r".*(ed|ing)$", st) or st in {"be","is","are","was","were","been","am","have","has","had","do","did","does"}:
                            has_verb = True

        bt_total = int(np.sum(between)) if between else 0
        bt_max   = int(np.max(between)) if between else 0
        bt_mean  = float(np.mean(between)) if between else 0.0

        return {
            "Pre_Tokens": int(pre),
            "Post_Tokens": int(post),
            "Between_Tokens_List": between,
            "Between_Tokens_Total": bt_total,
            "Between_Segments": int(len(between)),
            "Between_Max": bt_max,
            "Between_Mean": bt_mean,
            "Between_Has_Verb": bool(has_verb),
            "Between_Has_Subordinator": bool(has_sub)
        }

    def _pattern_label(self, span_labels):
        if not span_labels:
            return "none"
        labs = [l for l in span_labels]
        if labs == ["as_as"]:
            return "as_as"
        if labs.count("like") >= 2:
            return "like_like"
        if any(l.startswith("punc_") for l in labs):
            return "framed_multi" if len(labs) >= 2 else "framed_single"
        if len(labs) == 1:
            return "single"
        return "multi"

    # ----------------- Linguistic analysis (spaCy / fallback) -----------------
    def perform_comprehensive_linguistic_analysis(self):
        print("\nPERFORMING LINGUISTIC ANALYSIS (with pre/between/post segmentation)")
        print("-" * 60)
        if self.nlp is None:
            print("spaCy unavailable → simplified analysis (segmentation + sentiment only).")
            return self._perform_simplified_analysis()

        for name, df in list(self.datasets.items()):
            if df.empty:
                print(f"Skipping empty dataset: {name}")
                continue

            n = len(df)
            feats = {
                # --- segmentation block ---
                'Pre_Tokens': [0]*n,
                'Post_Tokens': [0]*n,
                'Between_Tokens_List_Raw': ["0"]*n,  # bracketless string; '0' if empty
                'Between_Tokens_List': ["0"]*n,      # user-facing same string
                'Between_Tokens_Total': [0]*n,
                'Between_Segments': [0]*n,
                'Between_Max': [0]*n,
                'Between_Mean': [0.0]*n,
                'Between_Has_Verb': [False]*n,
                'Between_Has_Subordinator': [False]*n,
                'Pre_Share': [np.nan]*n,
                'Between_Share': [np.nan]*n,
                'Post_Share': [np.nan]*n,
                'Comp_Count': [0]*n,
                'Comp_Labels': [json.dumps([])]*n,
                'Pattern_Label': ["none"]*n,

                # --- legacy / other features still in use downstream ---
                'Total_Tokens': [0]*n,
                'Pre_Post_Ratio': [np.nan]*n,
                'Lemmatized_Text': [None]*n,
                'POS_Tags': [None]*n,
                'POS_Distribution': [None]*n,
                'Sentiment_Polarity': [np.nan]*n,
                'Sentiment_Subjectivity': [np.nan]*n,
                'Comparative_Structure': [None]*n,
                'Syntactic_Complexity': [np.nan]*n,
                'Sentence_Length': [0]*n,
                'Adjective_Count': [0]*n,
                'Verb_Count': [0]*n,
                'Noun_Count': [0]*n,
                'Figurative_Density': [0.0]*n
            }

            for ridx, row in df.iterrows():
                sent = str(row.get('Sentence_Context', '') or '').strip()
                comp = row.get('Comparator_Type', '')
                if not sent:
                    continue
                try:
                    doc = self.nlp(sent)

                    # tokens for other features
                    tokens_nopunct = [t for t in doc if not t.is_space and not t.is_punct]
                    total = len(tokens_nopunct)

                    # --- multi-comparator detection & segmentation ---
                    spans = self._detect_comparator_spans(doc, use_spacy=True)
                    comp_labels = [lab for *_ab, lab in spans]
                    segs = self._segments_from_spans(doc, spans, use_spacy=True)

                    pre, post = segs["Pre_Tokens"], segs["Post_Tokens"]
                    bt_list = segs["Between_Tokens_List"]
                    bt_str = self._fmt_between_list(bt_list)
                    denom = pre + post + segs["Between_Tokens_Total"]
                    pre_share = (pre / denom) if denom else np.nan
                    between_share = (segs["Between_Tokens_Total"] / denom) if denom else np.nan
                    post_share = (post / denom) if denom else np.nan
                    pre_post_ratio = (pre / post) if post > 0 else np.nan
                    pattern = self._pattern_label(comp_labels)

                    loc = df.index.get_loc(ridx)

                    # Fills segmentation features
                    feats['Pre_Tokens'][loc] = pre
                    feats['Post_Tokens'][loc] = post
                    feats['Between_Tokens_List_Raw'][loc] = bt_str      # bracketless or '0'
                    feats['Between_Tokens_List'][loc] = bt_str
                    feats['Between_Tokens_Total'][loc] = segs["Between_Tokens_Total"]
                    feats['Between_Segments'][loc] = segs["Between_Segments"]
                    feats['Between_Max'][loc] = segs["Between_Max"]
                    feats['Between_Mean'][loc] = segs["Between_Mean"]
                    feats['Between_Has_Verb'][loc] = segs["Between_Has_Verb"]
                    feats['Between_Has_Subordinator'][loc] = segs["Between_Has_Subordinator"]
                    feats['Pre_Share'][loc] = pre_share
                    feats['Between_Share'][loc] = between_share
                    feats['Post_Share'][loc] = post_share
                    feats['Comp_Count'][loc] = len(spans)
                    feats['Comp_Labels'][loc] = json.dumps(comp_labels)
                    feats['Pattern_Label'][loc] = pattern

                    # legacy/other features
                    feats['Total_Tokens'][loc] = total
                    feats['Pre_Post_Ratio'][loc] = pre_post_ratio

                    lemmas = [t.lemma_.lower() for t in tokens_nopunct if not t.is_stop]
                    pos_tags = [t.pos_ for t in tokens_nopunct]
                    pos_dist = Counter(pos_tags)

                    # Sentiment (exploratory)
                    pol = subj = np.nan
                    if TextBlob is not None:
                        try:
                            blob = TextBlob(sent)
                            pol, subj = blob.sentiment.polarity, blob.sentiment.subjectivity
                        except Exception:
                            pass

                    # shallow structure summary
                    comp_struct = {
                        'has_explicit_comparator': any(t.text.lower() in ['like','as','than'] for t in doc),
                        'comparator_type': str(comp).strip() or "Unknown",
                        'comparative_adjectives': [t.text for t in doc if t.tag_ in ['JJR','RBR']],
                        'superlative_adjectives': [t.text for t in doc if t.tag_ in ['JJS','RBS']],
                        'modal_verbs': [t.text for t in doc if t.pos_ == 'AUX' and t.text.lower() in ['might','could','would','should','may']],
                        'epistemic_markers': [t.text for t in doc if t.text.lower() in ['perhaps','maybe','possibly','apparently','seemingly']]
                    }

                    def depth(tok, d=0):
                        if not list(tok.children):
                            return d
                        return max(depth(ch, d+1) for ch in tok.children)
                    roots = [t for t in doc if t.head == t]
                    try:
                        complexity = max(depth(r) for r in roots) if roots else 0
                    except Exception:
                        complexity = np.nan

                    slen = total
                    adj = sum(1 for t in tokens_nopunct if t.pos_ == 'ADJ')
                    vrb = sum(1 for t in tokens_nopunct if t.pos_ == 'VERB')
                    nou = sum(1 for t in tokens_nopunct if t.pos_ == 'NOUN')

                    figurative_markers = {'like','as','such','seem','appear','resemble','as if','as though'}
                    fdens = (sum(1 for t in tokens_nopunct if t.text.lower() in figurative_markers) / total) if total else 0

                    feats['Lemmatized_Text'][loc] = ' '.join(lemmas)
                    feats['POS_Tags'][loc] = '; '.join(pos_tags)
                    feats['POS_Distribution'][loc] = json.dumps(dict(pos_dist))
                    feats['Sentiment_Polarity'][loc] = pol
                    feats['Sentiment_Subjectivity'][loc] = subj
                    feats['Comparative_Structure'][loc] = json.dumps(comp_struct)
                    feats['Syntactic_Complexity'][loc] = complexity
                    feats['Sentence_Length'][loc] = slen
                    feats['Adjective_Count'][loc] = adj
                    feats['Verb_Count'][loc] = vrb
                    feats['Noun_Count'][loc] = nou
                    feats['Figurative_Density'][loc] = fdens

                except Exception as e:
                    print(f"  Error in {name} row {ridx}: {e}")

            # Attaches features to DF
            for k, v in feats.items():
                df[k] = v

            self.linguistic_features[name] = feats
            self.datasets[name] = df
            print(f"Finished linguistic analysis for {name}.")

        print("All datasets processed.")

    def _perform_simplified_analysis(self):
        # Fallback when spaCy is unavailable: still do segmentation using simple tokens
        for name, df in list(self.datasets.items()):
            if df.empty or 'Sentence_Context' not in df.columns:
                continue

            n = len(df)
            # init output columns (same schema)
            df['Pre_Tokens'] = 0
            df['Post_Tokens'] = 0
            df['Between_Tokens_List_Raw'] = "0"
            df['Between_Tokens_List'] = "0"
            df['Between_Tokens_Total'] = 0
            df['Between_Segments'] = 0
            df['Between_Max'] = 0
            df['Between_Mean'] = 0.0
            df['Between_Has_Verb'] = False
            df['Between_Has_Subordinator'] = False
            df['Pre_Share'] = np.nan
            df['Between_Share'] = np.nan
            df['Post_Share'] = np.nan
            df['Comp_Count'] = 0
            df['Comp_Labels'] = json.dumps([])
            df['Pattern_Label'] = "none"

            df['Total_Tokens'] = 0
            df['Pre_Post_Ratio'] = np.nan
            df['Sentiment_Polarity'] = np.nan
            df['Sentiment_Subjectivity'] = np.nan
            df['Sentence_Length'] = 0
            df['Lemmatized_Text'] = None
            df['POS_Tags'] = None
            df['POS_Distribution'] = None
            df['Comparative_Structure'] = None
            df['Syntactic_Complexity'] = np.nan
            df['Adjective_Count'] = 0
            df['Verb_Count'] = 0
            df['Noun_Count'] = 0
            df['Figurative_Density'] = 0.0

            for ridx, row in df.iterrows():
                sent = str(row.get('Sentence_Context','') or '').strip()
                if not sent:
                    continue
                tokens = self._simple_tokens(sent)
                content = [t for t in tokens if not re.fullmatch(r"[:;,.\-\(\)…]", str(t))]
                total = len(content)

                # detect spans and segment
                spans = self._detect_comparator_spans(tokens, use_spacy=False)
                comp_labels = [lab for *_ab, lab in spans]
                segs = self._segments_from_spans(tokens, spans, use_spacy=False)

                pre, post = segs["Pre_Tokens"], segs["Post_Tokens"]
                bt_list = segs["Between_Tokens_List"]
                bt_str = self._fmt_between_list(bt_list)
                denom = pre + post + segs["Between_Tokens_Total"]
                pre_share = (pre / denom) if denom else np.nan
                between_share = (segs["Between_Tokens_Total"] / denom) if denom else np.nan
                post_share = (post / denom) if denom else np.nan
                pre_post_ratio = (pre / post) if post > 0 else np.nan
                pattern = self._pattern_label(comp_labels)

                df.at[ridx, 'Total_Tokens'] = total
                df.at[ridx, 'Sentence_Length'] = total
                df.at[ridx, 'Pre_Tokens'] = pre
                df.at[ridx, 'Post_Tokens'] = post
                df.at[ridx, 'Pre_Post_Ratio'] = pre_post_ratio
                df.at[ridx, 'Between_Tokens_List_Raw'] = bt_str   # bracketless or '0'
                df.at[ridx, 'Between_Tokens_List'] = bt_str
                df.at[ridx, 'Between_Tokens_Total'] = segs["Between_Tokens_Total"]
                df.at[ridx, 'Between_Segments'] = segs["Between_Segments"]
                df.at[ridx, 'Between_Max'] = segs["Between_Max"]
                df.at[ridx, 'Between_Mean'] = segs["Between_Mean"]
                df.at[ridx, 'Between_Has_Verb'] = segs["Between_Has_Verb"]
                df.at[ridx, 'Between_Has_Subordinator'] = segs["Between_Has_Subordinator"]
                df.at[ridx, 'Pre_Share'] = pre_share
                df.at[ridx, 'Between_Share'] = between_share
                df.at[ridx, 'Post_Share'] = post_share
                df.at[ridx, 'Comp_Count'] = len(spans)
                df.at[ridx, 'Comp_Labels'] = json.dumps(comp_labels)
                df.at[ridx, 'Pattern_Label'] = pattern

                # Sentiment if available
                if TextBlob is not None:
                    try:
                        blob = TextBlob(sent)
                        df.at[ridx, 'Sentiment_Polarity'] = blob.sentiment.polarity
                        df.at[ridx, 'Sentiment_Subjectivity'] = blob.sentiment.subjectivity
                    except Exception:
                        pass

            self.datasets[name] = df

        print("Simplified analysis complete.")

    # ----------------- Instance-aligned F1 metrics -----------------
    def _pair_rows_by_sentence(self, gold_df, pred_df, fuzzy_threshold=0.92):
        if gold_df.empty or pred_df.empty:
            return [], gold_df, pred_df

        gold_df = gold_df.copy()
        pred_df = pred_df.copy()
        gold_df["_norm_sent"] = gold_df["Sentence_Context"].map(self._normalize_sentence_for_match)
        pred_df["_norm_sent"] = pred_df["Sentence_Context"].map(self._normalize_sentence_for_match)

        exact_pairs, used_pred = [], set()
        pred_lookup = {}
        for j, s in pred_df["_norm_sent"].items():
            pred_lookup.setdefault(s, []).append(j)

        for i, s in gold_df["_norm_sent"].items():
            if s in pred_lookup:
                js = [jj for jj in pred_lookup[s] if jj not in used_pred]
                if js:
                    j = js[0]
                    used_pred.add(j)
                    exact_pairs.append((i, j))

        unmatched_gold = [i for i in gold_df.index if i not in {gi for gi,_ in exact_pairs}]
        unmatched_pred = [j for j in pred_df.index if j not in used_pred]

        fuzzy_pairs = []
        for i in unmatched_gold:
            best_j, best_r = None, 0.0
            gi = gold_df.at[i, "_norm_sent"]
            for j in unmatched_pred:
                r = SequenceMatcher(None, gi, pred_df.at[j, "_norm_sent"]).ratio()
                if r > best_r:
                    best_r = r; best_j = j
            if best_j is not None and best_r >= fuzzy_threshold:
                used_pred.add(best_j)
                fuzzy_pairs.append((i, best_j))

        pairs = exact_pairs + fuzzy_pairs
        return pairs, gold_df, pred_df

    def _compute_f1_from_pairs(self, gold_df, pred_df, pairs, category_col="Category_Framework"):
        cats = sorted(set(gold_df[category_col].astype(str)) | set(pred_df[category_col].astype(str)))
        TP = {c:0 for c in cats}; FP = {c:0 for c in cats}; FN = {c:0 for c in cats}

        matched_gold = set(i for i,_ in pairs)
        matched_pred = set(j for _,j in pairs)

        for i,j in pairs:
            g = str(gold_df.at[i, category_col])
            p = str(pred_df.at[j, category_col])
            if p == g: TP[g] += 1
            else: FP[p] += 1; FN[g] += 1

        for i in gold_df.index:
            if i not in matched_gold: FN[str(gold_df.at[i, category_col])] += 1
        for j in pred_df.index:
            if j not in matched_pred: FP[str(pred_df.at[j, category_col])] += 1

        metrics = {}; micro_tp = micro_fp = micro_fn = 0
        for c in cats:
            tp, fp, fn = TP[c], FP[c], FN[c]
            prec = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            rec  = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1   = (2*prec*rec)/(prec+rec) if (prec+rec) > 0 else 0.0
            metrics[c] = {"tp": tp, "fp": fp, "fn": fn, "precision": prec, "recall": rec, "f1": f1}
            micro_tp += tp; micro_fp += fp; micro_fn += fn

        micro_prec = micro_tp / (micro_tp + micro_fp) if (micro_tp + micro_fp) > 0 else 0.0
        micro_rec  = micro_tp / (micro_tp + micro_fn) if (micro_tp + micro_fn) > 0 else 0.0
        micro_f1   = (2*micro_prec*micro_rec)/(micro_prec+micro_rec) if (micro_prec+micro_rec) > 0 else 0.0
        macro_f1   = np.mean([m["f1"] for m in metrics.values()]) if metrics else 0.0
        return metrics, {"micro_precision": micro_prec, "micro_recall": micro_rec, "micro_f1": micro_f1, "macro_f1": macro_f1}

    def calculate_corrected_f1_scores(self, fuzzy_threshold=0.92):
        print("\nCALCULATING INSTANCE-ALIGNED F1 METRICS")
        print("-" * 44)
        print(f"Assumptions: sentence-level alignment (exact, then fuzzy ≥ {fuzzy_threshold}); category = 'Category_Framework'.")

        manual_df = self.datasets.get('manual', pd.DataFrame())
        if manual_df.empty or 'Sentence_Context' not in manual_df or 'Category_Framework' not in manual_df:
            print("F1 unavailable: manual annotations missing/invalid.")
            self.comparison_results['f1_analysis'] = None
            return None, None

        out = {}

        def eval_one(pred_df, pred_name):
            if pred_df.empty or 'Sentence_Context' not in pred_df or 'Category_Framework' not in pred_df:
                print(f"{pred_name}: dataset missing required columns.")
                return None
            pairs, gdf, pdf = self._pair_rows_by_sentence(manual_df, pred_df, fuzzy_threshold=fuzzy_threshold)
            metrics, overall = self._compute_f1_from_pairs(gdf, pdf, pairs)
            print(f"{pred_name}: pairs={len(pairs)}  micro-F1={overall['micro_f1']:.3f}  macro-F1={overall['macro_f1']:.3f}")
            return {"pairs": len(pairs), "category_metrics": metrics, "overall": overall}

        rb = self.datasets.get('rule_based', pd.DataFrame())
        nl = self.datasets.get('nlp', pd.DataFrame())

        out['rule_based_vs_manual'] = eval_one(rb, "Rule-Based vs Manual")
        out['nlp_vs_manual'] = eval_one(nl, "NLP vs Manual")

        self.comparison_results['f1_analysis'] = out
        primary = out['rule_based_vs_manual']['overall']['micro_f1'] if out.get('rule_based_vs_manual') else None
        return out, primary

    # ----------------- Save / Export -----------------
    def save_comprehensive_results(self, output_path="comprehensive_linguistic_analysis_corrected.csv"):
        print("\nSAVING COMPREHENSIVE RESULTS …")
        frames = []
        for name, df in self.datasets.items():
            if df is None or df.empty:
                continue
            d = df.copy()
            for col, default in [
                ('Original_Dataset', name),
                ('Instance_ID', None),
                ('Sentence_Context', None),
                ('Category_Framework', None),
                ('Comparator_Type', None)
            ]:
                if col not in d.columns:
                    d[col] = default

            if d['Instance_ID'].isna().any() or (not d['Instance_ID'].astype(str).is_unique):
                d = self._ensure_ids(d, name)

            base = ['Instance_ID','Original_Dataset','Sentence_Context','Category_Framework','Comparator_Type']
            others = [c for c in d.columns if c not in base]
            d = d[base + others]
            frames.append(d)

        if not frames:
            print("No data to save.")
            return pd.DataFrame()

        combined = pd.concat(frames, ignore_index=True)

        # Stable sort: Manual → Restrictive → Less-Restrictive PG → BNC
        order = {
            'Manual_CloseReading': 1,
            'Restrictive_Dubliners': 2,
            'NLP_LessRestrictive_PG': 3,
            'BNC_Baseline': 4
        }
        combined['__order__'] = combined['Original_Dataset'].map(order).fillna(99).astype(int)

        def _id_numeric_tail(x):
            m = re.search(r'(\d+)$', str(x))
            return int(m.group(1)) if m else 0

        combined = combined.sort_values(
            by=['__order__','Original_Dataset','Instance_ID'],
            key=lambda s: s.map(_id_numeric_tail) if s.name == 'Instance_ID' else s
        ).drop(columns='__order__')

        combined.to_csv(output_path, index=False)
        print(f"Saved: {output_path}")
        print("Integrity:",
              "missing Instance_ID =", combined['Instance_ID'].isna().sum(),
              "| missing Original_Dataset =", combined['Original_Dataset'].isna().sum(),
              "| rows =", len(combined))
        print("Environment (for reproducibility):", self.env_info)
        return combined


# ========= RUN THE PIPELINE (sets paths) =========
manual_file_content = "/content/All Similes - Dubliners cont.csv"
rule_based_file_content = "/content/dubliners_rulebased_extraction.csv"
nlp_file_content = "/content/dubliners_nlp_less_restrictive_extraction.csv"
bnc_file_content = "/content/bnc_processed_similes.csv"

comparator = ComprehensiveLinguisticComparator()
comparator.load_datasets(manual_file_content, rule_based_file_content, nlp_file_content, bnc_file_content)
comparator.perform_comprehensive_linguistic_analysis()
f1_analysis, primary_f1 = comparator.calculate_corrected_f1_scores()  # instance-aligned F1
results_df = comparator.save_comprehensive_results("comprehensive_linguistic_analysis.csv")

print("\nPIPELINE COMPLETED.")


COMPREHENSIVE LINGUISTIC COMPARISON OF FOUR SIMILE DATASETS (FIXED + SEGMENTS)
Dataset 1: Manual Annotations (Ground Truth - Close Reading)
Dataset 2: Rule-Based Extraction (Restrictive - Domain-Informed)
Dataset 3: NLP Extraction (Less-Restrictive - PG Dubliners)
Dataset 4: BNC Baseline Corpus (Standard English Reference)
spaCy pipeline loaded: en_core_web_sm
Environment: {'python': '3.12.11 (main, Jun  4 2025, 08:56:18) [GCC 11.4.0]', 'pandas': '2.2.2', 'numpy': '2.0.2', 'spacy': '3.8.7', 'textblob': 'n/a'}

LOADING DATASETS WITH FIXED ID HANDLING & EXPLICIT LABELS
----------------------------------------------------------------------
Loading manual annotations…
Loading rule-based (restrictive)…
Loading NLP (less-restrictive PG)…
Loading BNC baseline…
Standardizing column names & adding Dataset_Source…
Standardization complete.
Harmonizing Category_Framework labels…
Category harmonization complete.
      manual: rows= 184  missing_IDs=0  missing_Original_Dataset=0
  rule_based: rows=

# 7. Statistical Significance Testing
# 7.1 Multi-Group Comparative Analysis
The statistical analysis distinguishes between Joyce Manual, Joyce Restrictive, Joyce Less-Restrictive, and BNC subsets to provide granular assessment of methodological differences.

# 7.2 Robust Statistical Framework
Implementation includes:

Four-way chi-square analysis for categorical distribution testing
Newcombe-Wilson confidence intervals for two-proportion comparisons
Binomial testing against BNC reference proportions
Welch t-tests and Mann-Whitney U tests for continuous feature assessment

# 7.3 Topic Modeling Integration
Latent Dirichlet Allocation provides thematic analysis across all dataset subsets, revealing content-based distinctions complementing statistical findings.

In [8]:
# =============================================================================
# ROBUST STATISTICAL SIGNIFICANCE + TOPIC MODELLING (Joyce subsets vs BNC)
# =============================================================================

import os, json, time
import numpy as np
import pandas as pd

from math import asin, sqrt
from scipy.stats import chi2_contingency, mannwhitneyu, ttest_ind, binomtest, norm, fisher_exact
from scipy.spatial.distance import jensenshannon
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

try:
    from statsmodels.stats.proportion import proportions_ztest, confint_proportions_2indep
    _HAS_STATSMODELS = True
except Exception:
    _HAS_STATSMODELS = False

# ---------- Setup ----------
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

ts = time.strftime("%Y%m%d_%H%M%S")
out_dir = os.path.join("analysis_outputs")
os.makedirs(out_dir, exist_ok=True)

print("\nROBUST STATISTICAL ANALYSIS (Joyce subsets vs BNC)")
print("=" * 75)

# --- Sanity: results_df must exist from Cell 1 ---
if 'results_df' not in globals() or results_df is None or results_df.empty:
    raise RuntimeError("results_df not found or empty. Run Cell 1 first.")

# --- Group labels (from Cell 1 'Original_Dataset') ---
LABELS = {
    "Manual_CloseReading":      "Joyce_Manual",
    "Restrictive_Dubliners":    "Joyce_Restrictive",
    "NLP_LessRestrictive_PG":   "Joyce_LessRestrictive",
    "BNC_Baseline":             "BNC"
}

df = results_df.copy()
if "Original_Dataset" not in df.columns:
    raise RuntimeError("results_df is missing 'Original_Dataset'.")
if "Category_Framework" not in df.columns:
    raise RuntimeError("results_df is missing 'Category_Framework'.")

df["__Group__"] = df["Original_Dataset"].map(LABELS).fillna(df["Original_Dataset"])

# --- Category whitelist (Joycean_Quasi was merged → keep unified 'Quasi_Similes') ---
KNOWN_CATEGORIES = {
    'Standard',
    'Quasi_Similes',          # unified quasi-simile label
    'Joycean_Quasi_Fuzzy',
    'Joycean_Framed',
    'Joycean_Silent',
    'Uncategorized'
}

bad_mask = ~df["Category_Framework"].isin(KNOWN_CATEGORIES)
if bad_mask.any():
    dropped = int(bad_mask.sum())
    print(f"[WARN] Dropping {dropped} rows with unexpected Category_Framework values: "
          f"{sorted(df.loc[bad_mask, 'Category_Framework'].astype(str).unique())}")
    df = df.loc[~bad_mask].copy()

# --- Quick distribution sanity table (post-harmonisation) ---
dist = (df.pivot_table(index="Category_Framework",
                       columns="Original_Dataset",
                       values="Instance_ID",
                       aggfunc="count", fill_value=0)
          .assign(Total=lambda x: x.sum(1))
          .sort_values("Total", ascending=False))
print("\nCategory distribution after harmonisation (counts):")
print(dist)

# --- Split groups ---
groups = {
    "Joyce_Manual":          df[df["__Group__"]=="Joyce_Manual"],
    "Joyce_Restrictive":     df[df["__Group__"]=="Joyce_Restrictive"],
    "Joyce_LessRestrictive": df[df["__Group__"]=="Joyce_LessRestrictive"],
    "BNC":                   df[df["__Group__"]=="BNC"]
}
for gname, gdf in groups.items():
    print(f"{gname:22s}: {len(gdf)} rows")

# ---------- Helpers ----------
def p_adjust_bh(p):
    """Benjamini–Hochberg FDR for a 1D array-like of p-values."""
    p = np.asarray(p, dtype=float)
    n = p.size
    order = np.argsort(p)
    ranked = np.empty(n, dtype=float)
    cummin = 1.0
    for i, idx in enumerate(order[::-1], start=1):
        rank = n - i + 1
        val = p[idx] * n / rank
        cummin = min(cummin, val)
        ranked[idx] = cummin
    return np.minimum(ranked, 1.0)

def cramers_v(chi2, n, r, c):
    """Cramér's V for r x c table."""
    if n <= 0 or min(r, c) <= 1:
        return np.nan
    return sqrt(chi2 / (n * (min(r, c) - 1)))

def cohens_h(p1, p2):
    """Cohen's h for proportions."""
    p1 = min(max(float(p1), 0.0), 1.0)
    p2 = min(max(float(p2), 0.0), 1.0)
    return 2 * (asin(sqrt(p1)) - asin(sqrt(p2)))

def hedges_g(a, b):
    """Hedges' g (small-sample corrected Cohen's d)."""
    a = np.asarray(a, float); b = np.asarray(b, float)
    na, nb = len(a), len(b)
    if na < 2 or nb < 2:
        return np.nan
    s1 = np.var(a, ddof=1); s2 = np.var(b, ddof=1)
    pooled = ((na-1)*s1 + (nb-1)*s2) / (na+nb-2) if (na+nb-2) > 0 else np.nan
    if not np.isfinite(pooled) or pooled <= 0:
        return np.nan
    d = (np.mean(a) - np.mean(b)) / np.sqrt(pooled)
    J = 1 - (3 / (4*(na+nb) - 9)) if (na+nb) > 2 else 1.0
    return d * J

def cliffs_delta_from_u(a, b):
    """Cliff's delta via Mann–Whitney U (orientation A > B)."""
    a = pd.Series(a).dropna().to_numpy()
    b = pd.Series(b).dropna().to_numpy()
    if len(a) == 0 or len(b) == 0:
        return np.nan
    u_ab, _ = mannwhitneyu(a, b, alternative="greater")
    return (2 * u_ab) / (len(a)*len(b)) - 1

# ---------- 1) 4-way Chi-square on Category_Framework ----------
cats = sorted(KNOWN_CATEGORIES)
contingency_4way = pd.DataFrame(
    {
        "Joyce_Manual":          [groups["Joyce_Manual"]["Category_Framework"].value_counts().get(cat,0) for cat in cats],
        "Joyce_Restrictive":     [groups["Joyce_Restrictive"]["Category_Framework"].value_counts().get(cat,0) for cat in cats],
        "Joyce_LessRestrictive": [groups["Joyce_LessRestrictive"]["Category_Framework"].value_counts().get(cat,0) for cat in cats],
        "BNC":                   [groups["BNC"]["Category_Framework"].value_counts().get(cat,0) for cat in cats],
    },
    index=cats
)

sim_used = False
try:
    chi2_4, p_4, dof_4, exp_4 = chi2_contingency(contingency_4way, correction=False)
    if (np.asarray(exp_4) < 5).any():
        try:
            chi2_4, p_4, dof_4, exp_4 = chi2_contingency(
                contingency_4way, correction=False, simulate_pval=True, num_simulation=5000
            )
            sim_used = True
        except TypeError:
            pass
except Exception as e:
    raise RuntimeError(f"chi2_contingency failed: {e}")

N_total = contingency_4way.values.sum()
V_4 = cramers_v(chi2_4, N_total, *contingency_4way.shape)

print("\n4-way Chi-square on Category_Framework (Joyce subsets vs BNC):")
print(f"χ² = {chi2_4:.4f} | df = {dof_4} | p = {p_4:.6f} | Cramér’s V = {V_4:.3f} | Monte-Carlo={sim_used}")

# Save contingency + expected + standardized residuals + per-cell p-values (FDR)
path_cont_4 = os.path.join(out_dir, f"chi2_contingency_by_subset_{ts}.csv")
path_exp_4  = os.path.join(out_dir, f"chi2_expected_by_subset_{ts}.csv")
contingency_4way.to_csv(path_cont_4)
exp_df_4 = pd.DataFrame(exp_4, index=cats, columns=contingency_4way.columns)
exp_df_4.to_csv(path_exp_4)

pearson_z = (contingency_4way - exp_df_4) / np.sqrt(exp_df_4.replace(0, np.nan))
cell_pvals = pearson_z.applymap(lambda z: 2*norm.sf(abs(z)) if pd.notnull(z) else np.nan)

flat = cell_pvals.values.flatten()
mask = np.isfinite(flat)
adj = np.full_like(flat, np.nan, dtype=float)
if mask.any():
    adj[mask] = p_adjust_bh(flat[mask])
cell_pvals_adj = pd.DataFrame(adj.reshape(cell_pvals.shape), index=cell_pvals.index, columns=cell_pvals.columns)

path_resid_z = os.path.join(out_dir, f"chi2_pearson_z_by_subset_{ts}.csv")
path_resid_p = os.path.join(out_dir, f"chi2_cell_p_by_subset_{ts}.csv")
path_resid_padj = os.path.join(out_dir, f"chi2_cell_padj_BH_by_subset_{ts}.csv")
pearson_z.to_csv(path_resid_z)
cell_pvals.to_csv(path_resid_p)
cell_pvals_adj.to_csv(path_resid_padj)

# Print the strongest drivers (optional but helpful)
absz = pearson_z.abs().stack().sort_values(ascending=False)
print("\nTop 10 standardized residuals (|z|):")
for (cat, grp), z in absz.head(10).items():
    obs = contingency_4way.loc[cat, grp]
    exp = exp_df_4.loc[cat, grp]
    print(f"  {grp:22s} | {cat:20s}  z={z:6.2f}  obs={obs} exp={exp:.1f}")

# ---------- 2) Two-proportion tests (each Joyce subset vs BNC) ----------
print("\nTwo-proportion tests (Newcombe–Wilson) for each Joyce subset vs BNC:")
two_prop_rows = []
bnc_total = len(groups["BNC"])
bnc_counts = groups["BNC"]["Category_Framework"].value_counts()

for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
    subset_total = len(groups[subset])
    subset_counts = groups[subset]["Category_Framework"].value_counts()
    for cat in cats:
        cA = subset_counts.get(cat,0); nA = subset_total
        cB = bnc_counts.get(cat,0);    nB = bnc_total
        pA = cA/nA if nA>0 else 0.0
        pB = cB/nB if nB>0 else 0.0
        row = {"Comparison":f"{subset}_vs_BNC", "Subset":subset, "Category":cat,
               "count_A":cA, "n_A":nA, "prop_A":pA, "count_B":cB, "n_B":nB, "prop_B":pB,
               "cohens_h": float(cohens_h(pA, pB))}
        # Skip boundary cases where both groups are 0 or 1 for this category
        if (cA == 0 and cB == 0) or (cA == nA and cB == nB) or (nA == 0 or nB == 0):
            row.update({"z": np.nan, "p_value": 1.0, "CI_low": np.nan, "CI_up": np.nan})
            print(f"  {subset:22s} | {cat:20s} (both groups at boundary → skip) h={row['cohens_h']:.3f}")
            two_prop_rows.append(row); continue
        if _HAS_STATSMODELS:
            try:
                z, pz = proportions_ztest(np.array([cA,cB]), np.array([nA,nB]))
            except Exception:
                # Haldane–Anscombe continuity correction fallback
                pA_ha = (cA + 0.5) / (nA + 1)
                pB_ha = (cB + 0.5) / (nB + 1)
                se = np.sqrt(pA_ha*(1-pA_ha)/(nA+1) + pB_ha*(1-pB_ha)/(nB+1))
                z = (pA_ha - pB_ha) / se if se>0 else np.nan
                pz = 2*norm.sf(abs(z)) if np.isfinite(z) else np.nan
            ci_low, ci_up = (np.nan, np.nan)
            try:
                ci_low, ci_up = confint_proportions_2indep(cA, nA, cB, nB, method="newcombe")
            except Exception:
                pass
            row.update({"z":float(z), "p_value":float(pz), "CI_low":float(ci_low), "CI_up":float(ci_up)})
            print(f"  {subset:22s} | {cat:20s} z={z:6.3f} p={pz:.6g} CI[{ci_low:.3f},{ci_up:.3f}] h={row['cohens_h']:.3f}")
        else:
            row.update({"z":np.nan, "p_value":np.nan, "CI_low":np.nan, "CI_up":np.nan})
            print(f"  {subset:22s} | {cat:20s} (statsmodels unavailable → skipping z/CI) h={row['cohens_h']:.3f}")
        two_prop_rows.append(row)

two_prop_df = pd.DataFrame(two_prop_rows)
if "p_value" in two_prop_df.columns:
    two_prop_df["p_adj_BH"] = p_adjust_bh(two_prop_df["p_value"].fillna(1.0).to_numpy())
path_two_prop = os.path.join(out_dir, f"two_prop_newcombe_by_subset_{ts}.csv")
two_prop_df.to_csv(path_two_prop, index=False)

# ---------- 3) Binomial tests (subset vs BNC reference proportion) ----------
print("\nBinomial tests (each Joyce subset vs BNC category proportion):")
binom_rows = []
for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
    nA = len(groups[subset])
    subset_counts = groups[subset]["Category_Framework"].value_counts()
    for cat in cats:
        cA = subset_counts.get(cat,0)
        cB = bnc_counts.get(cat,0); nB = bnc_total
        p_ref = (cB/nB) if nB>0 else 0.0
        if nA>0 and 0 < p_ref < 1:
            bt = binomtest(cA, n=nA, p=p_ref)
            pv = bt.pvalue
            binom_rows.append({"Comparison":f"{subset}_vs_BNC", "Subset":subset, "Category":cat,
                               "count_A":cA, "n_A":nA, "p_ref_BNC":p_ref, "p_value":pv})
            print(f"  {subset:22s} | {cat:20s} {cA}/{nA} vs p_ref={p_ref:.4f} p={pv:.6g}")
        else:
            binom_rows.append({"Comparison":f"{subset}_vs_BNC", "Subset":subset, "Category":cat,
                               "count_A":cA, "n_A":nA, "p_ref_BNC":p_ref, "p_value":np.nan})

binom_df = pd.DataFrame(binom_rows)
if "p_value" in binom_df.columns:
    binom_df["p_adj_BH"] = p_adjust_bh(binom_df["p_value"].fillna(1.0).to_numpy())
path_binom = os.path.join(out_dir, f"binomial_tests_by_subset_{ts}.csv")
binom_df.to_csv(path_binom, index=False)

# ---------- 4) Continuous features (subset vs BNC) ----------
print("\nContinuous features (Welch t + Mann–Whitney U) each Joyce subset vs BNC:")
continuous_feats = ["Sentence_Length","Pre_Post_Ratio","Sentiment_Polarity","Sentiment_Subjectivity"]
cont_rows = []

for feat in continuous_feats:
    for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
        A = pd.to_numeric(groups[subset].get(feat, pd.Series(dtype=float)), errors="coerce").dropna()
        B = pd.to_numeric(groups["BNC"].get(feat, pd.Series(dtype=float)), errors="coerce").dropna()
        row = {"Feature":feat, "Comparison":f"{subset}_vs_BNC", "Subset":subset,
               "A_n":int(A.shape[0]), "B_n":int(B.shape[0])}
        if len(A)>10 and len(B)>10:
            t,p_t = ttest_ind(A,B,equal_var=False)
            u,p_u = mannwhitneyu(A,B,alternative="two-sided")
            row.update({
                "A_mean":float(A.mean()), "A_median":float(A.median()),
                "B_mean":float(B.mean()), "B_median":float(B.median()),
                "t_stat":float(t), "t_pvalue":float(p_t),
                "U_stat":float(u), "U_pvalue":float(p_u),
                "hedges_g": float(hedges_g(A, B)),
                "cliffs_delta": float(cliffs_delta_from_u(A, B))
            })
            print(f"  {feat:22s} | {subset:22s} t={t:7.3f} p={p_t:.6g} | U={u:9.1f} p={p_u:.6g} | g={row['hedges_g']:.3f} δ={row['cliffs_delta']:.3f}")
        cont_rows.append(row)

cont_df = pd.DataFrame(cont_rows)
if "t_pvalue" in cont_df.columns:
    cont_df["t_padj_BH"] = p_adjust_bh(cont_df["t_pvalue"].fillna(1.0).to_numpy())
if "U_pvalue" in cont_df.columns:
    cont_df["U_padj_BH"] = p_adjust_bh(cont_df["U_pvalue"].fillna(1.0).to_numpy())

path_cont = os.path.join(out_dir, f"continuous_tests_by_subset_{ts}.csv")
cont_df.to_csv(path_cont, index=False)

# ---------- 4B) Comparator-segment metrics (Pre / Between / Post) — integrated from results_df ----------
print("\nComparator-segment statistics (Pre / Between / Post) — integrated (no external CSV)")
seg_cols = [
    "Pre_Tokens","Between_Tokens_Total","Post_Tokens","Total_Tokens",
    "Pre_Share","Between_Share","Post_Share",
    "Comp_Count","Between_Segments","Between_Max","Between_Mean","Between_Has_Verb","Between_Has_Subordinator"
]

seg_rows = []
for feat in [c for c in seg_cols if c in df.columns]:
    for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
        A = pd.to_numeric(groups[subset].get(feat, pd.Series(dtype=float)), errors="coerce").dropna()
        B = pd.to_numeric(groups["BNC"].get(feat, pd.Series(dtype=float)), errors="coerce").dropna()
        row = {"Feature": feat, "Comparison": f"{subset}_vs_BNC", "Subset": subset,
               "A_n": int(A.shape[0]), "B_n": int(B.shape[0])}
        # For booleans (Has_Verb/Subordinator), treat as numeric 0/1
        if feat in ["Between_Has_Verb","Between_Has_Subordinator"]:
            A = A.astype(float); B = B.astype(float)
        if len(A) > 10 and len(B) > 10 and np.isfinite(A).any() and np.isfinite(B).any():
            t, p_t = ttest_ind(A, B, equal_var=False)
            u, p_u = mannwhitneyu(A, B, alternative="two-sided")
            row.update({
                "A_mean": float(A.mean()), "A_median": float(A.median()),
                "B_mean": float(B.mean()), "B_median": float(B.median()),
                "t_stat": float(t), "t_pvalue": float(p_t),
                "U_stat": float(u), "U_pvalue": float(p_u),
                "hedges_g": float(hedges_g(A, B)),
                "cliffs_delta": float(cliffs_delta_from_u(A, B))
            })
            print(f"  {feat:20s} | {subset:22s} t={t:7.3f} p={p_t:.3g} | U={u:9.1f} p={p_u:.3g} | g={row['hedges_g']:.3f} δ={row['cliffs_delta']:.3f}")
        seg_rows.append(row)

seg_cont_df = pd.DataFrame(seg_rows)
if "t_pvalue" in seg_cont_df.columns:
    seg_cont_df["t_padj_BH"] = p_adjust_bh(seg_cont_df["t_pvalue"].fillna(1.0).to_numpy())
if "U_pvalue" in seg_cont_df.columns:
    seg_cont_df["U_padj_BH"] = p_adjust_bh(seg_cont_df["U_pvalue"].fillna(1.0).to_numpy())

path_seg_cont = os.path.join(out_dir, f"continuous_segment_tests_by_subset_{ts}.csv")
seg_cont_df.to_csv(path_seg_cont, index=False)

# ---- (ii) Does 'Between > 0' differ by group? (χ² with robust fallbacks) ----
col_between = df.get("Between_Tokens_Total", pd.Series(index=df.index)).fillna(0)
tmp = df.assign(Seg_Between_gt0=(pd.to_numeric(col_between, errors="coerce").fillna(0) > 0).astype(int))

pres = tmp.groupby("__Group__")["Seg_Between_gt0"].agg(["sum","count"])
order = [g for g in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"] if g in pres.index]
pres = pres.loc[order]

presence_tbl = pd.DataFrame({
    "Between>0": pres["sum"].astype(int),
    "Between=0": (pres["count"] - pres["sum"]).astype(int)
})

print("\nBetween-clause presence × Group (χ²):")
print(presence_tbl if not presence_tbl.empty else "(no usable data)")

if presence_tbl.shape[0] >= 2 and presence_tbl.shape[1] == 2:
    exp_seg = None
    try:
        chi2_seg, p_seg, dof_seg, exp_seg = chi2_contingency(presence_tbl, correction=False)
        V_seg = cramers_v(chi2_seg, presence_tbl.values.sum(), *presence_tbl.shape)
        print(f"  χ² = {chi2_seg:.3f} | df = {dof_seg} | p = {p_seg:.6f} | Cramér’s V = {V_seg:.3f}")
    except ValueError:
        try:
            chi2_seg, p_seg, dof_seg, exp_seg = chi2_contingency(
                presence_tbl, correction=False, simulate_pval=True, num_simulation=5000
            )
            V_seg = cramers_v(chi2_seg, presence_tbl.values.sum(), *presence_tbl.shape)
            print(f"  Monte-Carlo χ² = {chi2_seg:.3f} | df = {dof_seg} | p = {p_seg:.6f} | V = {V_seg:.3f}")
        except TypeError:
            print("  χ² not valid (zero expected). Pairwise Fisher exact p-values vs BNC:")
            for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
                if subset in presence_tbl.index and "BNC" in presence_tbl.index:
                    tbl2 = presence_tbl.loc[[subset, "BNC"]].to_numpy()
                    _, p_f = fisher_exact(tbl2)  # two-sided
                    print(f"   - {subset:22s}: p = {p_f:.6g}")
            p_seg = np.nan; chi2_seg = np.nan; dof_seg = 1
    # Save presence table + expected + residuals if we have them
    path_presence_csv = os.path.join(out_dir, f"between_presence_by_group_{ts}.csv")
    presence_tbl.to_csv(path_presence_csv)
    if 'exp_seg' in locals() and exp_seg is not None:
        path_presence_exp = os.path.join(out_dir, f"between_presence_expected_by_group_{ts}.csv")
        path_presence_resid = os.path.join(out_dir, f"between_presence_residuals_by_group_{ts}.csv")
        exp_df = pd.DataFrame(exp_seg, index=presence_tbl.index, columns=presence_tbl.columns)
        exp_df.to_csv(path_presence_exp)
        resid = (presence_tbl - exp_df) / np.sqrt(exp_df.replace(0, np.nan))
        resid.to_csv(path_presence_resid)
    else:
        path_presence_exp = path_presence_resid = None
else:
    print("  Not enough non-zero groups/columns for χ²; skipping.")
    path_presence_csv = path_presence_exp = path_presence_resid = None

# ---------- 4C) Pattern/shaping statistics (anchors, patterns, between diagnostics, declared comparator) ----------
print("\nPattern/shaping statistics (anchors, patterns, between diagnostics, declared comparator)")

work = df.copy()

# Parse Comparative_Structure JSON safely
def _as_dict(x):
    if isinstance(x, dict):
        return x
    if isinstance(x, str) and x.strip():
        try:
            return json.loads(x)
        except Exception:
            return {}
    return {}

comp_struct = work.get("Comparative_Structure", pd.Series(index=work.index)).apply(_as_dict)

# Helper: safe Sentence_Context string
def _sent(x):
    return str(x) if pd.notnull(x) else ""

# Flags from Comparative_Structure
work["has_modal"] = comp_struct.apply(lambda d: bool(d.get("modal_verbs")))
work["has_epistemic_marker"] = comp_struct.apply(lambda d: bool(d.get("epistemic_markers")))

# Declared comparator (normalized)
decl = work.get("Comparator_Type", pd.Series(index=work.index)).astype(str).str.strip().str.lower()

def starts(d, pfx): return d.fillna("").str.startswith(pfx)
def contains(d, sub): return d.fillna("").str.contains(sub, regex=False)

work["declared_comp_like"]       = (decl == "like")
work["declared_comp_as"]         = (decl == "as")
work["declared_comp_as_if"]      = ((decl == "as if") | (decl == "as-if"))
work["declared_comp_as_though"]  = ((decl == "as though") | (decl == "as-though"))
work["declared_comp_resemble"]   = starts(decl, "resembl")
work["declared_comp_seem"]       = starts(decl, "seem")
work["declared_comp_appear"]     = starts(decl, "appear")
work["declared_comp_colon"]      = ((decl == "colon") | (decl == ":"))
work["declared_comp_semicolon"]  = ((decl == "semicolon") | (decl == ";"))
work["declared_comp_dash"]       = ((decl == "en dash") | (decl == "–") | (decl == "—") | (decl == "-"))
work["declared_comp_ellipsis"]   = ((decl == "ellipsis") | (decl == "...") | (decl == "…"))

# Anchor detections from text (heuristic + declared comparator)
sent_series = work.get("Sentence_Context", pd.Series(index=work.index)).apply(_sent).str.lower().fillna("")

# as…as heuristic: at least two 'as' tokens separated within a small window OR explicit 'as ... as'
def detect_as_as(s):
    if not s: return False
    if "as ... as" in s: return True
    toks = [t for t in re.split(r"\W+", s) if t]
    positions = [i for i,t in enumerate(toks) if t == "as"]
    for i in range(len(positions)):
        for j in range(i+1, len(positions)):
            if 1 <= (positions[j] - positions[i]) <= 8:
                return True
    return False

import re
work["anchor_like"]          = work["declared_comp_like"] | sent_series.str.contains(r"\blike\b", regex=True)
work["anchor_as_if"]         = work["declared_comp_as_if"] | sent_series.str.contains(r"\bas if\b", regex=True)
work["anchor_as_though"]     = work["declared_comp_as_though"] | sent_series.str.contains(r"\bas though\b", regex=True)
work["anchor_as_as"]         = work["declared_comp_as"] | sent_series.apply(detect_as_as)
work["anchor_resemble_family"]= work["declared_comp_resemble"] | sent_series.str.contains(r"\bresembl\w*\b", regex=True)
work["anchor_seem_family"]    = work["declared_comp_seem"] | sent_series.str.contains(r"\bseem\w*\b", regex=True)
work["anchor_appear_family"]  = work["declared_comp_appear"] | sent_series.str.contains(r"\bappear\w*\b", regex=True)
work["anchor_colon"]          = work["declared_comp_colon"] | sent_series.str.contains(r":", regex=False)
work["anchor_semicolon"]      = work["declared_comp_semicolon"] | sent_series.str.contains(r";", regex=False)
work["anchor_dash"]           = work["declared_comp_dash"] | sent_series.str.contains(r"[–—-]", regex=True)
work["anchor_ellipsis"]       = work["declared_comp_ellipsis"] | sent_series.str.contains(r"\.\.\.|…", regex=True)

# Between diagnostics from integrated metrics
work["between_has_verb"]        = work.get("Between_Has_Verb", pd.Series(index=work.index)).fillna(0).astype(bool)
work["between_has_subordinator"]= work.get("Between_Has_Subordinator", pd.Series(index=work.index)).fillna(0).astype(bool)

# Compact "signature" string per row: which anchors are present (sorted, joined)
anchor_cols = [
    "anchor_like","anchor_as_as","anchor_as_if","anchor_as_though",
    "anchor_resemble_family","anchor_seem_family","anchor_appear_family",
    "anchor_colon","anchor_semicolon","anchor_dash","anchor_ellipsis"
]
def signature(row):
    on = [c.replace("anchor_","") for c in anchor_cols if bool(row.get(c, False))]
    return ",".join(sorted(on)) if on else "none"
work["anchor_signature"] = work[anchor_cols].apply(signature, axis=1)

# ---- Pattern flag tests (each flag proportion vs BNC) ----
flag_cols = [
    # anchors
    "anchor_like","anchor_as_as","anchor_as_if","anchor_as_though",
    "anchor_resemble_family","anchor_seem_family","anchor_appear_family",
    "anchor_colon","anchor_semicolon","anchor_dash","anchor_ellipsis",
    # between diagnostics
    "between_has_verb","between_has_subordinator",
    # declared comparator families (sanity)
    "declared_comp_like","declared_comp_as","declared_comp_as_if","declared_comp_as_though",
    "declared_comp_resemble","declared_comp_seem","declared_comp_appear",
    "declared_comp_colon","declared_comp_semicolon","declared_comp_dash","declared_comp_ellipsis",
]

flag_rows = []
for flag in [f for f in flag_cols if f in work.columns]:
    bnc = work[work["__Group__"]=="BNC"][flag].astype(bool)
    nB = bnc.shape[0]; cB = int(bnc.sum())
    for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive"]:
        sub = work[work["__Group__"]==subset][flag].astype(bool)
        nA = sub.shape[0]; cA = int(sub.sum())
        pA = cA/nA if nA>0 else 0.0
        pB = cB/nB if nB>0 else 0.0
        row = {"Flag":flag, "Subset":subset, "count_A":cA, "n_A":nA, "prop_A":pA, "count_B":cB, "n_B":nB, "prop_B":pB}
        if (nA>0 and nB>0) and not ((cA==0 and cB==0) or (cA==nA and cB==nB)):
            if _HAS_STATSMODELS:
                try:
                    z, pz = proportions_ztest(np.array([cA,cB]), np.array([nA,nB]))
                except Exception:
                    pA_ha = (cA + 0.5) / (nA + 1)
                    pB_ha = (cB + 0.5) / (nB + 1)
                    se = np.sqrt(pA_ha*(1-pA_ha)/(nA+1) + pB_ha*(1-pB_ha)/(nB+1))
                    z = (pA_ha - pB_ha) / se if se>0 else np.nan
                    pz = 2*norm.sf(abs(z)) if np.isfinite(z) else np.nan
                row.update({"z": float(z), "p_value": float(pz)})
            else:
                row.update({"z": np.nan, "p_value": np.nan})
        else:
            row.update({"z": np.nan, "p_value": 1.0})
        flag_rows.append(row)

flag_df = pd.DataFrame(flag_rows)
if "p_value" in flag_df.columns:
    flag_df["p_adj_BH"] = p_adjust_bh(flag_df["p_value"].fillna(1.0).to_numpy())

path_flag = os.path.join(out_dir, f"pattern_flag_tests_by_subset_{ts}.csv")
flag_df.to_csv(path_flag, index=False)

# ---- Top anchor signatures per group ----
sig_rows = []
for g in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"]:
    w = work[work["__Group__"]==g]
    vc = w["anchor_signature"].value_counts()
    total = int(vc.sum())
    for s, c in vc.items():
        sig_rows.append({"Group": g, "Anchor_Signature": s, "Count": int(c), "Prop": (c/total if total>0 else 0.0)})
top_sigs_df = pd.DataFrame(sig_rows).sort_values(["Group","Count"], ascending=[True,False])

path_sigs = os.path.join(out_dir, f"top_anchor_signatures_by_group_{ts}.csv")
top_sigs_df.to_csv(path_sigs, index=False)

# ---- Console summary for pattern flags (BH<0.05) ----
sig = flag_df.copy()
if "p_adj_BH" in sig.columns:
    sig = sig[sig["p_adj_BH"] < 0.05].copy()
    if not sig.empty:
        sig["abs_diff"] = (sig["prop_A"] - sig["prop_B"]).abs()
        sig = sig.sort_values(["p_adj_BH","abs_diff"], ascending=[True, False])
        print("\nSignificant pattern flags (BH<0.05):")
        for _, r in sig.iterrows():
            d = (r["prop_A"] - r["prop_B"])
            print(f"  {r['Subset']:22s} | {r['Flag']:28s} Δ={d:+.3f} "
                  f"(A={r['prop_A']:.3f}, B={r['prop_B']:.3f}) p_adj={r['p_adj_BH']:.3g}")
    else:
        print("\nSignificant pattern flags (BH<0.05): none")

# ---- Quick console view of top anchor signatures per group ----
print("\nTop anchor signatures per group (n≤10 each):")
for g in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"]:
    sub = top_sigs_df[top_sigs_df["Group"]==g].head(10)
    if not sub.empty:
        print(f"\n{g}")
        for _, row in sub.iterrows():
            print(f"  {row['Anchor_Signature']:<40s} count={row['Count']:>3d}  prop={row['Prop']:.3f}")

# ---- Anchor-signature divergence across groups (Jensen–Shannon) ----
all_sigs = sorted(work["anchor_signature"].dropna().unique())
def dist_for(g):
    vc = work[work["__Group__"]==g]["anchor_signature"].value_counts()
    p = np.array([vc.get(s,0) for s in all_sigs], dtype=float)
    return p/p.sum() if p.sum()>0 else np.ones(len(all_sigs))/len(all_sigs)

G = ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"]
js_rows = []
for i in range(len(G)):
    for j in range(i+1,len(G)):
        # squared Jensen–Shannon divergence for [0,1] interpretability
        js = float(jensenshannon(dist_for(G[i]), dist_for(G[j]))**2)
        js_rows.append({"Group_A":G[i], "Group_B":G[j], "JSD":js})
js_df = pd.DataFrame(js_rows).sort_values("JSD", ascending=False)
js_path = os.path.join(out_dir, f"anchor_signature_js_divergence_{ts}.csv")
js_df.to_csv(js_path, index=False)
print("\nAnchor-signature divergence (Jensen–Shannon, higher = more different):")
print(js_df.to_string(index=False))

# ---------- 5) Topic modelling (per subset + BNC) ----------
print("\nTOPIC MODELLING (per subset + BNC)")

def prepare_corpus(gdf):
    """Prefer Lemmatized_Text from Cell 1; fallback to Sentence_Context."""
    if "Lemmatized_Text" in gdf.columns and gdf["Lemmatized_Text"].notna().any():
        texts = gdf["Lemmatized_Text"].fillna("").astype(str).tolist()
    else:
        texts = gdf["Sentence_Context"].fillna("").astype(str).tolist() if "Sentence_Context" in gdf.columns else []
    return [t for t in texts if t.strip()]

def lda_topics(corpus, n_topics=5, n_top_words=10, max_df=0.85, min_df=2, max_features=5000, random_state=RANDOM_STATE):
    if not corpus:
        return None, None, None
    vectorizer = CountVectorizer(
        max_df=max_df, min_df=min_df, max_features=max_features,
        stop_words="english",
        token_pattern=r"(?u)\b[^\W\d_][^\W\d_]+\b"
    )
    X = vectorizer.fit_transform(corpus)
    lda = LatentDirichletAllocation(
        n_components=n_topics, random_state=random_state, learning_method="batch"
    )
    lda.fit(X)
    terms = vectorizer.get_feature_names_out()
    topics = []
    for comp in lda.components_:
        top_idx = comp.argsort()[:-n_top_words-1:-1]
        topics.append([terms[i] for i in top_idx])
    doc_topic = lda.transform(X)  # rows sum ~1
    return topics, doc_topic, terms

topics_summary = {"params":{"n_topics":5,"n_top_words":10,"vectorizer":"CountVectorizer"}, "groups":{}}
topic_rows = []
topicmix_rows = []

for subset in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"]:
    corpus = prepare_corpus(groups[subset])
    if corpus:
        tpcs, doc_topic, terms = lda_topics(corpus, n_topics=5, n_top_words=10)
        topics_summary["groups"][subset] = tpcs if tpcs is not None else []
        if tpcs:
            for i, words in enumerate(tpcs, 1):
                topic_rows.append({"Group":subset, "Topic":i, "Top_Words":", ".join(words)})
            topic_means = doc_topic.mean(axis=0) if doc_topic is not None else None
            if topic_means is not None:
                for i, val in enumerate(topic_means, 1):
                    topicmix_rows.append({"Group":subset, "Topic":i, "Mean_Weight":float(val)})
        print(f"  Topics generated for {subset}: {len(tpcs) if tpcs else 0}")
    else:
        topics_summary["groups"][subset] = []
        print(f"  Not enough text for {subset}")

topics_json_path = os.path.join(out_dir, f"lda_topics_by_subset_{ts}.json")
with open(topics_json_path, "w", encoding="utf-8") as f:
    json.dump(topics_summary, f, ensure_ascii=False, indent=2)

topics_csv = pd.DataFrame(topic_rows, columns=["Group","Topic","Top_Words"])
topics_csv_path = os.path.join(out_dir, f"lda_topics_by_subset_{ts}.csv")
topics_csv.to_csv(topics_csv_path, index=False)

topicmix_csv = pd.DataFrame(topicmix_rows, columns=["Group","Topic","Mean_Weight"])
topicmix_csv_path = os.path.join(out_dir, f"lda_topic_mix_by_subset_{ts}.csv")
topicmix_csv.to_csv(topicmix_csv_path, index=False)

# ---------- 6) Optional robustness: 3-way χ² ----------
try:
    cont_3 = contingency_4way.drop(columns=["Joyce_LessRestrictive"])
    chi2_3, p_3, dof_3, _ = chi2_contingency(cont_3, correction=False)
    V_3 = cramers_v(chi2_3, cont_3.values.sum(), *cont_3.shape)
    print(f"\n3-way χ² (Manual/Restrictive/BNC): χ²={chi2_3:.2f} df={dof_3} p={p_3:.3g} V={V_3:.3f}")
except Exception:
    pass

# ---------- 7) Master summary JSON ----------
master = {
    "generated_at": ts,
    "random_state": RANDOM_STATE,
    "note": "By-subset outputs (Manual / Restrictive / Less-Restrictive vs BNC) with unified Quasi_Similes label and FDR corrections.",
    "chi_square_4way": {
        "chi2": float(chi2_4), "dof": int(dof_4), "p_value": float(p_4),
        "cramers_v": float(V_4), "monte_carlo": bool(sim_used),
        "N": int(N_total)
    },
    "files": {
        "chi2_contingency_by_subset_csv": path_cont_4,
        "chi2_expected_by_subset_csv": path_exp_4,
        "chi2_pearson_z_by_subset_csv": path_resid_z,
        "chi2_cell_p_by_subset_csv": path_resid_p,
        "chi2_cell_padj_BH_by_subset_csv": path_resid_padj,
        "two_prop_newcombe_by_subset_csv": path_two_prop,
        "binomial_tests_by_subset_csv": path_binom,
        "continuous_tests_by_subset_csv": path_cont,
        "continuous_segment_tests_by_subset_csv": path_seg_cont,
        "between_presence_by_group_csv": path_presence_csv,
        "between_presence_expected_by_group_csv": path_presence_exp,
        "between_presence_residuals_by_group_csv": path_presence_resid,
        "pattern_flag_tests_by_subset_csv": path_flag,
        "top_anchor_signatures_by_group_csv": path_sigs,
        "anchor_signature_js_divergence_csv": js_path,
        "lda_topics_by_subset_json": topics_json_path,
        "lda_topics_by_subset_csv": topics_csv_path,
        "lda_topic_mix_by_subset_csv": topicmix_csv_path
    }
}
master_path = os.path.join(out_dir, f"stats_and_topics_summary_by_subset_{ts}.json")
with open(master_path, "w", encoding="utf-8") as f:
    json.dump(master, f, ensure_ascii=False, indent=2)

print("\nSAVED OUTPUTS (by-subset)")
print(" - 4-way contingency:", master["files"]["chi2_contingency_by_subset_csv"])
print(" - 4-way expected:", master["files"]["chi2_expected_by_subset_csv"])
print(" - 4-way Pearson z:", master["files"]["chi2_pearson_z_by_subset_csv"])
print(" - 4-way cell p-values:", master["files"]["chi2_cell_p_by_subset_csv"])
print(" - 4-way cell p-values (BH):", master["files"]["chi2_cell_padj_BH_by_subset_csv"])
print(" - Two-proportion (subset vs BNC):", master["files"]["two_prop_newcombe_by_subset_csv"])
print(" - Binomial (subset vs BNC):", master["files"]["binomial_tests_by_subset_csv"])
print(" - Continuous tests (subset vs BNC):", master["files"]["continuous_tests_by_subset_csv"])
print(" - Segment tests (subset vs BNC):", master["files"]["continuous_segment_tests_by_subset_csv"])
print(" - Between presence by group:", master["files"]["between_presence_by_group_csv"])
print(" - Pattern flags (subset vs BNC):", master["files"]["pattern_flag_tests_by_subset_csv"])
print(" - Top anchor signatures:", master["files"]["top_anchor_signatures_by_group_csv"])
print(" - Anchor-signature divergence:", master["files"]["anchor_signature_js_divergence_csv"])
print(" - Topics JSON (per subset):", master["files"]["lda_topics_by_subset_json"])
print(" - Topics CSV (per subset):", master["files"]["lda_topics_by_subset_csv"])
print(" - Topic mix CSV (per subset):", master["files"]["lda_topic_mix_by_subset_csv"])
print(" - Master summary JSON:", master_path)
print("\nDONE.")



ROBUST STATISTICAL ANALYSIS (Joyce subsets vs BNC)

Category distribution after harmonisation (counts):
Original_Dataset     BNC_Baseline  Manual_CloseReading  \
Category_Framework                                       
Standard                      118                   93   
Quasi_Similes                  82                   53   
Joycean_Quasi_Fuzzy             0                   13   
Joycean_Framed                  0                   18   
Joycean_Silent                  0                    6   
Uncategorized                   0                    1   

Original_Dataset     NLP_LessRestrictive_PG  Restrictive_Dubliners  Total  
Category_Framework                                                         
Standard                                330                    150    691  
Quasi_Similes                             0                     47    182  
Joycean_Quasi_Fuzzy                       0                     14     27  
Joycean_Framed                            0       

# 8. Academic Reporting and Documentation

## 8.1 Professional Report Generation
The HTML report generator produces a polished, self-contained document suitable for:
- **Peer review** and journal **supplementary materials**
- Internal/external **research documentation** and **reproducibility** records
- **Academic presentations** (slides/posters) and dissemination to non-specialists

It compiles results directly from the timestamped CSV/JSON artifacts written to `analysis_outputs/`, ensuring the report is traceable to concrete files.

## 8.2 Results Integration
The report synthesizes all major components of the analysis, including:
- **Instance-aligned F1** summaries (exact + fuzzy sentence matching) for extractor vs manual annotations
- **4-way χ²** contingency results with **expected counts**, **standardized residuals (z)**, and **BH-FDR** per-cell p-values
- **Two-proportion tests** (Newcombe CIs) with **Cohen’s h** effect sizes, BH-FDR corrected
- **Binomial tests** vs BNC reference proportions, BH-FDR corrected
- **Continuous feature** comparisons (Welch *t*, Mann–Whitney *U*) with **Hedges’ g**, **Cliff’s δ**, and optional **Hodges–Lehmann** shifts (bootstrap CIs)
- **Topic modelling** per subset (LDA top words) and **mean topic-mix** weights
- **Dataset and category overviews**, reflecting the **harmonised taxonomy** with **Quasi_Similes** as the unified label

## 8.3 Academic Standards and Transparency
The report follows scholarly communication norms:
- Clear sectioning, professional typography, descriptive figure/table captions
- Explicit **statistical choices** (e.g., BH-FDR; Monte-Carlo χ² fallback when expected counts are low)
- **Environment stamping** (Python/library versions), deterministic seeds, and a **file manifest** with timestamps for reproducibility
- **Taxonomy harmonisation** note (Joycean_Quasi → **Quasi_Similes**) to prevent label inflation
- Methodological caveats (e.g., **TextBlob sentiment** reported as exploratory; the **less-restrictive NLP** subset being **all Standard** is treated as a control and excluded in robustness checks when appropriate)

## 8.4 Reproducibility & Data Availability (recommended)
- Archive the full `analysis_outputs/` directory, `comprehensive_linguistic_analysis_corrected.csv`, and this notebook.
- Include licensing/usage notes for external corpora (e.g., **BNC** access terms) and cite the **Project Gutenberg** source for *Dubliners*.
- Provide a short **README** describing how to re-run Cells 1–3 and regenerate the identical HTML using the saved artifacts.


In [9]:
# =============================================================================
# ACADEMIC HTML REPORT GENERATOR — UPDATED (data-driven, harmonised labels)
# Generates a comprehensive, reproducible HTML report from Cells 1–2 outputs
# =============================================================================

import os
import glob
import json
import pandas as pd
from datetime import datetime
import numpy as np

print("GENERATING ACADEMIC HTML REPORT")
print("=" * 50)

# Timestamp for report
now = datetime.now()
report_timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
report_date = now.strftime("%Y%m%d_%H%M%S")

# ---------- Helpers ----------
def latest_file(pattern, base="analysis_outputs"):
    files = glob.glob(os.path.join(base, pattern))
    return max(files, key=os.path.getctime) if files else None

def safe_read_csv(path):
    if not path or not os.path.exists(path):
        return pd.DataFrame()
    try:
        return pd.read_csv(path)
    except Exception as e:
        print(f"[WARN] Could not load CSV {path}: {e}")
        return pd.DataFrame()

def safe_read_json(path):
    if not path or not os.path.exists(path):
        return {}
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception as e:
        print(f"[WARN] Could not load JSON {path}: {e}")
        return {}

def df_for_html(df, index=False, max_rows=None):
    if df is None or df.empty:
        return pd.DataFrame()
    d = df.copy()
    if max_rows is not None and len(d) > max_rows:
        d = d.head(max_rows)
        d.__truncated__ = True
    return d

def create_table_html(df, title="", max_rows=20, index=False):
    """Create HTML table with styling; auto-handle empty and truncation note."""
    if df is None or df.empty:
        return f"<p><em>No data available for {title}</em></p>"
    d = df.copy()
    trunc = False
    if max_rows and len(d) > max_rows:
        d = d.head(max_rows)
        trunc = True
    if index:
        d = d.reset_index()
    table_html = d.to_html(classes='analysis-table', escape=False, index=False)
    note = f"<p class='truncated-note'><em>Showing first {max_rows} of {len(df)} rows</em></p>" if trunc else ""
    return f"""
    <div class="table-container">
        <h4>{title}</h4>
        <div class="table-wrapper">{table_html}</div>
        {note}
    </div>
    """

# ---------- Pull in analysis outputs from Cell 2 ----------
out_dir = "analysis_outputs"
os.makedirs(out_dir, exist_ok=True)

paths = {
    # Chi-square + per-cell
    "chi2_cont": latest_file("chi2_contingency_by_subset_*.csv", out_dir),
    "chi2_exp": latest_file("chi2_expected_by_subset_*.csv", out_dir),
    "chi2_z": latest_file("chi2_pearson_z_by_subset_*.csv", out_dir),
    "chi2_p": latest_file("chi2_cell_p_by_subset_*.csv", out_dir),
    "chi2_padj": latest_file("chi2_cell_padj_BH_by_subset_*.csv", out_dir),
    # Proportions / binomial
    "two_prop": latest_file("two_prop_newcombe_by_subset_*.csv", out_dir),
    "binom": latest_file("binomial_tests_by_subset_*.csv", out_dir),
    # Continuous (legacy)
    "cont": latest_file("continuous_tests_by_subset_*.csv", out_dir),
    # NEW: Comparator-segment metrics & presence
    "seg_cont": latest_file("continuous_segment_tests_by_subset_*.csv", out_dir),
    "presence": latest_file("between_presence_by_group_*.csv", out_dir),
    # NEW: Pattern flags & anchor signatures
    "pattern_flags": latest_file("pattern_flag_tests_by_subset_*.csv", out_dir),
    "top_sigs": latest_file("top_anchor_signatures_by_group_*.csv", out_dir),
    "js_div": latest_file("anchor_signature_js_divergence_*.csv", out_dir),
    # Topics
    "topics": latest_file("lda_topics_by_subset_*.csv", out_dir),
    "topicmix": latest_file("lda_topic_mix_by_subset_*.csv", out_dir),
    # HL shifts if present
    "hl": latest_file("continuous_HL_shifts_*.csv", out_dir),
    # Master summary
    "master": latest_file("stats_and_topics_summary_by_subset_*.json", out_dir),
}

chi2_cont_df = safe_read_csv(paths["chi2_cont"])
chi2_exp_df  = safe_read_csv(paths["chi2_exp"])
chi2_z_df    = safe_read_csv(paths["chi2_z"])
chi2_p_df    = safe_read_csv(paths["chi2_p"])
chi2_padj_df = safe_read_csv(paths["chi2_padj"])
two_prop_df  = safe_read_csv(paths["two_prop"])
binom_df     = safe_read_csv(paths["binom"])
cont_df      = safe_read_csv(paths["cont"])
seg_cont_df  = safe_read_csv(paths["seg_cont"])
presence_df  = safe_read_csv(paths["presence"])
pattern_df   = safe_read_csv(paths["pattern_flags"])
top_sigs_df  = safe_read_csv(paths["top_sigs"])
js_div_df    = safe_read_csv(paths["js_div"])
topics_df    = safe_read_csv(paths["topics"])
topicmix_df  = safe_read_csv(paths["topicmix"])
hl_df        = safe_read_csv(paths["hl"])
master_json  = safe_read_json(paths["master"])

# ---------- Summaries from Cell 1 (results_df, f1_analysis, comparator.env_info) ----------
def create_summary_stats_html():
    if 'results_df' not in globals() or results_df is None or results_df.empty:
        return "<p><em>No results data available</em></p>"
    dcounts = results_df['Original_Dataset'].value_counts()
    ccounts = results_df['Category_Framework'].value_counts()

    items_d = "".join([f"<li><strong>{k}:</strong> {int(v):,} instances</li>" for k,v in dcounts.items()])
    items_c = "".join([f"<li><strong>{k}:</strong> {int(v):,} ({(v/len(results_df))*100:.1f}%)</li>" for k,v in ccounts.items()])

    return f"""
    <div class="summary-stats">
        <div class="stat-group">
            <h4>Dataset Distribution</h4>
            <ul>{items_d}</ul>
            <p><strong>Total Instances:</strong> {len(results_df):,}</p>
        </div>
        <div class="stat-group">
            <h4>Category Distribution</h4>
            <ul>{items_c}</ul>
        </div>
    </div>
    """

def f1_summary_html():
    micro_rb = macro_rb = micro_nlp = macro_nlp = None
    pairs_rb = pairs_nlp = None
    if 'f1_analysis' in globals() and isinstance(f1_analysis, dict):
        rb = f1_analysis.get('rule_based_vs_manual') or f1_analysis.get('Rule-Based vs Manual')
        nl = f1_analysis.get('nlp_vs_manual') or f1_analysis.get('NLP vs Manual')
        if rb and rb.get("overall"):
            micro_rb = rb["overall"].get("micro_f1")
            macro_rb = rb["overall"].get("macro_f1")
            pairs_rb = rb.get("pairs")
        if nl and nl.get("overall"):
            micro_nlp = nl["overall"].get("micro_f1")
            macro_nlp = nl["overall"].get("macro_f1")
            pairs_nlp = nl.get("pairs")
    elif 'comparator' in globals():
        try:
            fa = comparator.comparison_results.get('f1_analysis', {})
            rb = fa.get('rule_based_vs_manual')
            nl = fa.get('nlp_vs_manual')
            if rb and rb.get("overall"):
                micro_rb = rb["overall"].get("micro_f1")
                macro_rb = rb["overall"].get("macro_f1")
                pairs_rb = rb.get("pairs")
            if nl and nl.get("overall"):
                micro_nlp = nl["overall"].get("micro_f1")
                macro_nlp = nl["overall"].get("macro_f1")
                pairs_nlp = nl.get("pairs")
        except Exception:
            pass

    if micro_rb is None and micro_nlp is None:
        return "<p><em>F1 metrics unavailable (run Cell 1 in this session).</em></p>"

    lines = []
    if micro_rb is not None:
        lines.append(f"• Rule-Based vs Manual: micro-F1 = {micro_rb:.3f}, macro-F1 = {macro_rb:.3f}{' (pairs=' + str(pairs_rb) + ')' if pairs_rb else ''}")
    if micro_nlp is not None:
        lines.append(f"• NLP vs Manual: micro-F1 = {micro_nlp:.3f}, macro-F1 = {macro_nlp:.3f}{' (pairs=' + str(pairs_nlp) + ')' if pairs_nlp else ''}")

    return f"""
    <div class="highlight">
      <strong>F1 Summary:</strong><br>
      {'<br>'.join(lines)}
      <br>• Total instances processed: {len(results_df):,} (across all datasets)
    </div>
    """

def env_html():
    info = {}
    if 'comparator' in globals():
        try:
            info = comparator.env_info
        except Exception:
            info = {}
    if not info:
        return ""
    items = "".join([f"<li><strong>{k}:</strong> {v}</li>" for k,v in info.items()])
    return f"""
    <div class="methodology">
      <strong>Environment:</strong>
      <ul>{items}</ul>
    </div>
    """

# ---------- Build analytic highlight snippets ----------
def chi2_summary_html():
    if not master_json:
        return ""
    chi = master_json.get("chi_square_4way", {})
    chi_line = (f"χ² = {chi.get('chi2', float('nan')):.3f} | df = {chi.get('dof', 0)} | "
                f"p = {chi.get('p_value', float('nan')):.3g} | Cramér’s V = {chi.get('cramers_v', float('nan')):.3f} "
                f"| Monte-Carlo={chi.get('monte_carlo', False)} | N={chi.get('N', 0)}")
    return f"<p>{chi_line}</p>"

def top_residuals_html(k=10):
    if chi2_z_df.empty or chi2_cont_df.empty or chi2_exp_df.empty:
        return ""
    z_long = chi2_z_df.copy()
    z_long = z_long.rename(columns={"Unnamed: 0":"Category_Framework"}) if "Unnamed: 0" in z_long.columns else z_long
    z_long = z_long.melt(id_vars=[c for c in ["Category_Framework"] if c in z_long.columns],
                         var_name="Group", value_name="z")
    z_long["abs_z"] = z_long["z"].abs()
    cont = chi2_cont_df.copy()
    cont = cont.rename(columns={"Unnamed: 0":"Category_Framework"}) if "Unnamed: 0" in cont.columns else cont
    exp = chi2_exp_df.copy()
    exp = exp.rename(columns={"Unnamed: 0":"Category_Framework"}) if "Unnamed: 0" in exp.columns else exp
    obs_long = cont.melt(id_vars=[c for c in ["Category_Framework"] if c in cont.columns],
                         var_name="Group", value_name="obs")
    exp_long = exp.melt(id_vars=[c for c in ["Category_Framework"] if c in exp.columns],
                        var_name="Group", value_name="exp")
    m = (z_long.merge(obs_long, on=["Category_Framework","Group"], how="left")
               .merge(exp_long, on=["Category_Framework","Group"], how="left"))
    m = m.sort_values("abs_z", ascending=False).head(k)
    return create_table_html(m, f"Top {k} standardized residuals (|z|)", max_rows=k)

def sig_two_prop_html(alpha=0.05, top=15):
    if two_prop_df.empty or "p_adj_BH" not in two_prop_df.columns:
        return ""
    df = two_prop_df.copy()
    df["|h|"] = df.get("cohens_h", 0).abs()
    sig = df[df["p_adj_BH"] < alpha].sort_values(["p_adj_BH","|h|"])
    cols = [c for c in ["Subset","Category","prop_A","prop_B","z","p_value","p_adj_BH","cohens_h"] if c in sig.columns]
    return create_table_html(sig[cols], f"Two-proportion (BH<{alpha}) — top {top}", max_rows=top)

def sig_binom_html(alpha=0.05, top=15):
    if binom_df.empty or "p_adj_BH" not in binom_df.columns:
        return ""
    df = binom_df.copy()
    sig = df[df["p_adj_BH"] < alpha].sort_values("p_adj_BH")
    cols = [c for c in ["Subset","Category","count_A","n_A","p_ref_BNC","p_value","p_adj_BH"] if c in sig.columns]
    return create_table_html(sig[cols], f"Binomial vs BNC (BH<{alpha}) — top {top}", max_rows=top)

def sig_continuous_html(alpha=0.05):
    if cont_df.empty:
        return ""
    d = cont_df.copy()
    if "U_padj_BH" in d.columns:
        sig = d[d["U_padj_BH"] < alpha].sort_values("U_padj_BH")
        label = "Mann–Whitney U (BH)"
        padj_col = "U_padj_BH"
    elif "t_padj_BH" in d.columns:
        sig = d[d["t_padj_BH"] < alpha].sort_values("t_padj_BH")
        label = "Welch t (BH)"
        padj_col = "t_padj_BH"
    else:
        return ""
    if sig.empty:
        return "<p><em>No continuous feature differences were significant after FDR correction.</em></p>"
    cols = [c for c in ["Feature","Subset","A_n","B_n","A_mean","B_mean","A_median","B_median",
                        "t_stat","t_pvalue","U_stat","U_pvalue","hedges_g","cliffs_delta", padj_col] if c in sig.columns]
    return create_table_html(sig[cols], f"Continuous features — significant by {label}", max_rows=15)

# NEW: significant segment metrics (Pre/Between/Post family)
def sig_seg_continuous_html(alpha=0.05):
    if seg_cont_df.empty:
        return ""
    d = seg_cont_df.copy()
    padj_col = "U_padj_BH" if "U_padj_BH" in d.columns else ("t_padj_BH" if "t_padj_BH" in d.columns else None)
    if not padj_col:
        return ""
    sig = d[d[padj_col] < alpha].copy()
    if sig.empty:
        return "<p><em>No segment metric differences were significant after FDR correction.</em></p>"
    # Order: strongest adjusted p then effect size magnitude if available
    if "hedges_g" in sig.columns:
        sig["|g|"] = sig["hedges_g"].abs()
        sig = sig.sort_values([padj_col,"|g|"])
    else:
        sig = sig.sort_values(padj_col)
    keep = [c for c in ["Feature","Subset","A_n","B_n","A_mean","B_mean","A_median","B_median",
                        "t_stat","t_pvalue","U_stat","U_pvalue","hedges_g","cliffs_delta", padj_col] if c in sig.columns]
    return create_table_html(sig[keep], f"Comparator-segment metrics — significant (BH<{alpha})", max_rows=20)

# NEW: pattern flags (BH<0.05) table
def sig_pattern_flags_html(alpha=0.05, top=25):
    if pattern_df.empty or "p_adj_BH" not in pattern_df.columns:
        return ""
    d = pattern_df.copy()
    d["abs_diff"] = (d["prop_A"] - d["prop_B"]).abs()
    sig = d[d["p_adj_BH"] < alpha].sort_values(["p_adj_BH","abs_diff"])
    keep = [c for c in ["Subset","Flag","prop_A","prop_B","z","p_value","p_adj_BH"] if c in sig.columns]
    return create_table_html(sig[keep], f"Pattern/anchor flags — significant (BH<{alpha}) — top {top}", max_rows=top)

# NEW: top anchor signatures per group (show top 10 each)
def top_anchor_signatures_html():
    if top_sigs_df.empty:
        return ""
    # For compactness, keep top 10 per group
    parts = []
    for g in ["Joyce_Manual","Joyce_Restrictive","Joyce_LessRestrictive","BNC"]:
        sub = top_sigs_df[top_sigs_df["Group"]==g]
        if sub.empty:
            continue
        sub = sub.sort_values("Count", ascending=False).head(10)
        parts.append(create_table_html(sub, f"Top anchor signatures — {g}", max_rows=10))
    return "".join(parts) if parts else ""

# NEW: JS divergence (anchor-signature distributions)
def anchor_js_divergence_html():
    if js_div_df.empty:
        return ""
    d = js_div_df.copy().sort_values("JSD", ascending=False)
    return create_table_html(d, "Anchor-signature divergence (Jensen–Shannon; higher = more different)", max_rows=12)

# ---------- Build the HTML ----------
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
<meta charset="UTF-8" />
<meta name="viewport" content="width=device-width, initial-scale=1.0"/>
<title>Joyce Simile Research: Comprehensive Linguistic Analysis Report</title>
<style>
    body {{
        font-family: 'Times New Roman', serif;
        line-height: 1.6;
        margin: 0;
        padding: 20px;
        background-color: #f9f9f9;
        color: #333;
    }}
    .container {{
        max-width: 1200px;
        margin: 0 auto;
        background: white;
        padding: 30px;
        border-radius: 8px;
        box-shadow: 0 2px 10px rgba(0,0,0,0.1);
    }}
    .header {{
        text-align: center;
        border-bottom: 3px solid #2c3e50;
        padding-bottom: 20px;
        margin-bottom: 30px;
    }}
    .header h1 {{ color: #2c3e50; margin: 0; font-size: 2.2em; font-weight: bold; }}
    .header .subtitle {{ color: #7f8c8d; font-size: 1.1em; margin: 10px 0 5px 0; font-style: italic; }}
    .header .timestamp {{ color: #95a5a6; font-size: 0.9em; }}
    .section {{
        margin: 30px 0;
        padding: 20px;
        border-left: 4px solid #3498db;
        background-color: #f8f9fa;
    }}
    .section h2 {{
        color: #2c3e50;
        margin-top: 0; border-bottom: 2px solid #ecf0f1; padding-bottom: 10px;
    }}
    .section h3 {{ color: #34495e; margin-top: 25px; }}
    .section h4 {{ color: #5d6d7e; margin-top: 20px; margin-bottom: 10px; }}
    .analysis-table {{ width: 100%; border-collapse: collapse; margin: 15px 0; font-size: 0.9em; }}
    .analysis-table th {{ background:#34495e; color:#fff; padding:12px 8px; text-align:left; font-weight:bold; }}
    .analysis-table td {{ padding:10px 8px; border-bottom:1px solid #ddd; }}
    .analysis-table tr:nth-child(even) {{ background-color:#f2f2f2; }}
    .analysis-table tr:hover {{ background-color:#e8f4fd; }}
    .summary-stats {{ display:grid; grid-template-columns:1fr 1fr; gap:30px; margin:20px 0; }}
    .stat-group {{ background:#fff; padding:20px; border-radius:6px; border:1px solid #e1e8ed; }}
    .stat-group h4 {{ margin-top:0; color:#2c3e50; border-bottom:1px solid #ecf0f1; padding-bottom:8px; }}
    .stat-group ul {{ list-style-type:none; padding:0; }}
    .stat-group li {{ padding:5px 0; border-bottom:1px solid #f8f9fa; }}
    .highlight {{ background:#d1ecf1; padding:15px; border-left:4px solid #17a2b8; margin:15px 0; }}
    .methodology {{ background:#f8f9fa; padding:15px; border-radius:5px; margin:15px 0; font-style:italic; }}
    .table-container {{ margin:20px 0; }}
    .table-wrapper {{ overflow-x:auto; }}
    .truncated-note {{ color:#6c757d; font-size:0.9em; margin-top:5px; }}
    .footer {{ text-align:center; margin-top:40px; padding-top:20px; border-top:2px solid #ecf0f1; color:#7f8c8d; font-size:0.9em; }}
    @media (max-width: 768px) {{
        .summary-stats {{ grid-template-columns:1fr; }}
        .container {{ padding:15px; }}
        .analysis-table {{ font-size:0.85em; }}
    }}
</style>
</head>
<body>
<div class="container">
    <div class="header">
        <h1>Joyce Simile Research</h1>
        <div class="subtitle">Comprehensive Linguistic Analysis Report</div>
        <div class="subtitle">Manual Annotations vs Extraction Methods vs BNC Baseline</div>
        <div class="timestamp">Generated on {report_timestamp}</div>
    </div>

    <div class="section">
        <h2>Executive Summary</h2>
        <p>This report compiles the results of a comparative analysis of simile usage in <em>Dubliners</em>, spanning manual expert annotations, a restrictive rule-based extractor, a less-restrictive NLP extractor, and a BNC baseline. Categories were harmonised with <strong>Quasi_Similes</strong> as the unified label for the Joyce/BNC quasi-simile phenomenon.</p>
        {chi2_summary_html()}
    </div>

    <div class="section">
        <h2>Dataset Overview</h2>
        <p>Four datasets were analysed, representing complementary identification approaches and a baseline:</p>
        {create_summary_stats_html()}
        {env_html()}
        <div class="methodology">
            <strong>Methodology:</strong> Features include token counts, comparator-segment metrics (Pre / Between / Post), pre/post comparator ratio, POS distribution, syntactic complexity, TextBlob sentiment (exploratory), and comparative structure flags. Between spans are computed across detected comparator anchors (e.g., <em>as … as</em>, <em>like</em>, framing punctuation); we report totals and ratios, assess group differences with non-parametric tests, and a χ² test on presence (Between &gt; 0).
        </div>
    </div>

    <div class="section">
        <h2>Performance Metrics</h2>
        <h3>Instance-Aligned F1 Scores</h3>
        {f1_summary_html()}
    </div>

    <div class="section">
        <h2>Statistical Results</h2>
        <h3>4-way Category Distribution</h3>
"""

# 4-way chi-square tables
if not chi2_cont_df.empty:
    html_content += create_table_html(chi2_cont_df, "Observed counts by group × category", max_rows=10, index=True)
if not chi2_exp_df.empty:
    html_content += create_table_html(chi2_exp_df, "Expected counts under independence", max_rows=10, index=True)
if not chi2_z_df.empty:
    html_content += create_table_html(chi2_z_df, "Pearson standardized residuals (z)", max_rows=10, index=True)
    html_content += top_residuals_html(k=10)
if not chi2_padj_df.empty:
    html_content += create_table_html(chi2_padj_df, "Per-cell p-values (BH-adjusted)", max_rows=10, index=True)

# Two-proportion
if not two_prop_df.empty:
    html_content += """
        <h3>Two-Proportion Tests (Joyce subsets vs BNC)</h3>
        <p>Newcombe confidence intervals and Cohen’s h. BH correction applied across tests.</p>
    """
    if "p_adj_BH" not in two_prop_df.columns and "p_value" in two_prop_df.columns:
        pv = two_prop_df["p_value"].fillna(1.0).to_numpy()
        order = np.argsort(pv)
        ranked = np.empty_like(pv, dtype=float)
        cummin = 1.0
        n = pv.size
        for i, idx in enumerate(order[::-1], start=1):
            rank = n - i + 1
            val = pv[idx] * n / rank
            cummin = min(cummin, val)
            ranked[idx] = min(cummin, 1.0)
        two_prop_df["p_adj_BH"] = ranked
    html_content += create_table_html(two_prop_df, "All two-proportion results", max_rows=15)
    html_content += sig_two_prop_html(alpha=0.05, top=15)

# Binomial
if not binom_df.empty:
    if "p_adj_BH" not in binom_df.columns and "p_value" in binom_df.columns:
        pv = binom_df["p_value"].fillna(1.0).to_numpy()
        order = np.argsort(pv)
        ranked = np.empty_like(pv, dtype=float)
        cummin = 1.0
        n = pv.size
        for i, idx in enumerate(order[::-1], start=1):
            rank = n - i + 1
            val = pv[idx] * n / rank
            cummin = min(cummin, val)
            ranked[idx] = min(cummin, 1.0)
        binom_df["p_adj_BH"] = ranked
    html_content += """
        <h3>Binomial Tests (Joyce subsets vs BNC reference proportion)</h3>
        <p>One-sample tests per category per subset; BH correction across tests.</p>
    """
    html_content += create_table_html(binom_df, "All binomial results", max_rows=12)
    html_content += sig_binom_html(alpha=0.05, top=12)

# Continuous (legacy features)
if not cont_df.empty:
    html_content += """
        <h3>Continuous Features</h3>
        <p>Welch t and Mann–Whitney U with effect sizes (Hedges’ g, Cliff’s δ). BH correction applied.</p>
    """
    html_content += create_table_html(cont_df, "Continuous feature comparisons", max_rows=12)
    html_content += sig_continuous_html(alpha=0.05)
if not hl_df.empty:
    html_content += create_table_html(hl_df, "Hodges–Lehmann location shifts (A − B) with bootstrap CIs", max_rows=12)

# NEW: Comparator-segment metrics & presence
html_content += """
    <h3>Comparator-Segment Metrics (Pre / Between / Post)</h3>
    <p>Integrated from Cell 1; totals and shares per segment, plus between-span diagnostics.</p>
"""
if not seg_cont_df.empty:
    html_content += create_table_html(seg_cont_df, "Segment metrics — subset vs BNC (Welch t / U, with BH)", max_rows=20)
    html_content += sig_seg_continuous_html(alpha=0.05)

if not presence_df.empty:
    html_content += create_table_html(presence_df, "Between-clause presence × Group (counts)", max_rows=10)

# NEW: Pattern/anchor flags & signatures
html_content += """
    <h3>Comparator Anchors & Pattern Flags</h3>
    <p>Binary flags for anchors (e.g., <em>like</em>, <em>as…as</em>, punctuation) and between-span diagnostics; two-proportion tests vs BNC with BH correction.</p>
"""
if not pattern_df.empty:
    html_content += create_table_html(pattern_df, "All pattern/anchor flag tests", max_rows=20)
    html_content += sig_pattern_flags_html(alpha=0.05, top=25)

if not top_sigs_df.empty:
    html_content += top_anchor_signatures_html()
if not js_div_df.empty:
    html_content += anchor_js_divergence_html()

# Topics
html_content += """
    </div>
    <div class="section">
        <h2>Topic Modelling</h2>
        <p>Per-subset LDA topics (CountVectorizer; 5 topics × 10 words). Topic mixture weights summarised by mean.</p>
"""
if not topics_df.empty:
    html_content += create_table_html(topics_df, "Top words per topic × group", max_rows=20)
if not topicmix_df.empty:
    html_content += create_table_html(topicmix_df, "Mean topic weights per group", max_rows=20)

# Sample of comprehensive results
if 'results_df' in globals() and isinstance(results_df, pd.DataFrame) and not results_df.empty:
    sample_cols = [c for c in ['Instance_ID','Original_Dataset','Category_Framework','Comparator_Type',
                               'Sentence_Length','Sentiment_Polarity','Pre_Share','Between_Share','Post_Share',
                               'Comp_Count','Between_Segments','Between_Max','Between_Mean'] if c in results_df.columns]
    sample_results = results_df.head(25)[sample_cols].round(3)
    html_content += f"""
    </div>
    <div class="section">
        <h2>Comprehensive Results — Sample</h2>
        <p>Representative snippet of the full dataset (first 25 rows):</p>
        {create_table_html(sample_results, "Sample of comprehensive analysis", max_rows=25)}
        <div class="methodology">
            The full dataset includes lemmatisation, POS distributions, syntactic complexity, comparator anchors, and comparator-segment diagnostics.
        </div>
    </div>
    """

# Files list
files_list = []
files_list.append("comprehensive_linguistic_analysis_corrected.csv")
for k, p in paths.items():
    if p:
        files_list.append(os.path.relpath(p))
files_items = "".join([f"<li><code>{f}</code></li>" for f in sorted(set(files_list))])

html_content += f"""
    <div class="section">
        <h2>Files Generated</h2>
        <ul>{files_items}</ul>
    </div>

    <div class="footer">
        <p>Generated by Comprehensive Linguistic Analysis Pipeline</p>
        <p>Joyce Simile Research Project • {report_timestamp}</p>
        <p><em>Categories harmonised with unified <strong>Quasi_Similes</strong>; results computed using FDR corrections where applicable.</em></p>
    </div>
</div>
</body>
</html>
"""

# Save HTML report
report_filename = f"joyce_simile_analysis_report_{report_date}.html"
with open(report_filename, 'w', encoding='utf-8') as f:
    f.write(html_content)

print(f"✓ Academic HTML report generated: {report_filename}")
print(f"✓ File size: {os.path.getsize(report_filename):,} bytes")
print(f"✓ Report length: {len(html_content):,} characters")
print("\nREPORT READY — open in a browser to view.")


GENERATING ACADEMIC HTML REPORT
✓ Academic HTML report generated: joyce_simile_analysis_report_20250831_105405.html
✓ File size: 78,719 bytes
✓ Report length: 78,665 characters

REPORT READY — open in a browser to view.


# 9. Research Implications and Future Directions

## 9.1 Computational Literary Analysis
Using sentence-level **instance-aligned evaluation** (exact match, then fuzzy ≥ 0.92), the extractors partially recover the manual categories:
- **Rule-Based vs Manual:** micro-F1 ≈ **0.343**, macro-F1 ≈ **0.178** (pairs matched: see Cell 1 output)
- **NLP (Less-Restrictive) vs Manual:** micro-F1 ≈ **0.292**, macro-F1 ≈ **0.059**

These scores indicate **meaningful but incomplete replication** of expert judgments—appropriate for unsupervised/heuristic extractors—and motivate targeted improvements (Section 9.4). Importantly, categorical distributions still differ strongly across corpora (**χ²≈281.88**, **Cramér’s V≈0.318**, *p*≪.001), so the **signal is primarily categorical**, not scalar.

**Key distributional contrasts (harmonised labels):**
- **BNC**: *Quasi_Similes* ≈ **41%** (82/200) vs **Joyce-Manual** ≈ **29%** (53/184).  
- **Joyce-Manual** shows enrichments in **Joycean_Framed**, **Joycean_Quasi_Fuzzy**, and **Joycean_Silent**.
- **NLP (Less-Restrictive)** is **all Standard** (330/330), by design—useful as a control but excluded in robustness checks where appropriate.

## 9.2 Innovation Quantification
Under the unified scheme:
- **Joycean-specific categories** (Framed + Quasi_Fuzzy + Silent) account for **~20%** of **Joyce-Manual** (37/184).
- **Quasi_Similes** appear **less** in **Joyce-Manual** (~29%) relative to **BNC** (~41%), with significant two-proportion differences (BH-corrected; moderate effects, e.g., |*h*|≈0.26–0.42 for Manual/Restrictive vs BNC).

This suggests Joyce’s corpus **rebalances** the space of similes: fewer conventional quasi-similes than BNC and **more Joycean-specific forms**, evidencing stylistic innovation distinct from baseline English.

## 9.3 Methodological Contributions
- **Label harmonisation**: *Joycean_Quasi* merged into **Quasi_Similes** to align BNC and Joyce phenomena, avoiding double-counting and artificial χ² inflation.
- **Robust inference**: 4-way χ² with per-cell residuals and **BH-FDR**; two-proportion tests with **Newcombe CIs** + **Cohen’s h**; binomial tests vs BNC reference; Welch-*t* & Mann–Whitney-*U* with **Hedges’ g** and **Cliff’s δ**; optional 3-way χ² excluding the degenerate NLP group.
- **Instance-aligned evaluation**: sentence-pairing (exact+fuzzy) for extractor vs manual categories, a more faithful alternative to bag-of-counts F1.
- **Reproducibility**: environment stamping; deterministic random seeds; saved CSV/JSON artifacts.

## 9.4 Future Directions
1. **Improve extractor recall** for Joycean categories  
   - Add patterns for **punctuation-mediated** comparators (colon/semicolon/ellipsis) and **‘as … as’** spans.  
   - Incorporate dependency features (governor-comparator-target) and discourse cues.

2. **Supervised modelling**  
   - Train a classifier on paired manual/extractor sentences (use current pairs as silver labels), with features from syntax, lemmata, and comparator spans.

3. **Error analysis & ablations**  
   - Per-category confusion and residuals to identify systematic misses; ablate comparator detection vs category mapping.

4. **Topic–category linkage**  
   - Relate LDA topic mixtures to categories (e.g., Framed vs Standard) and test with permutation or regression.

5. **Generalisation**  
   - Validate on other Joyce texts or contemporaries; check stability of the Joycean category enrichments.

---

## References and Data Sources

**Primary Text**  
- Joyce, James. *Dubliners*. Project Gutenberg.

**Baseline Corpus**  
- British National Corpus (BNC), used as a **standard English** reference. *Ensure appropriate licensing/access for the specific BNC source employed.*

---

## Computational Tools
- **spaCy** (`en_core_web_sm`): tokenisation, POS, dependencies  
- **scikit-learn**: CountVectorizer, LDA, utilities  
- **SciPy**: χ², *t*-tests, Mann–Whitney U, binomial  
- **statsmodels** (if available): two-proportion z-tests, Newcombe CIs  
- **TextBlob**: exploratory sentiment  
- **pandas / numpy**: data handling and numerics

---

## Research Framework (Summary)
- **Evaluation**: instance-aligned F1 (micro/macro) via sentence pairing (exact + fuzzy ≥ 0.92).  
- **Categorical inference**: 4-way χ² with per-cell residual z and BH-FDR; two-proportion tests (Newcombe CIs, Cohen’s h); binomial tests vs BNC proportions.  
- **Continuous inference**: Welch-*t* and Mann–Whitney-*U* with **Hedges’ g**, **Cliff’s δ**, and **Hodges–Lehmann** shifts (bootstrap CIs).  
- **Topics**: LDA per subset using **CountVectorizer** (5 topics × 10 words), with mean topic-mix summaries.  
- **Harmonised taxonomy**: **Quasi_Similes** (unified), **Joycean_Framed**, **Joycean_Quasi_Fuzzy**, **Joycean_Silent**, **Standard**, **Uncategorized**.

> **Terminology note:** *Quasi_Similes* is the unified tag covering both the BNC’s “Quasi_Similes” and Joyce’s former “Joycean_Quasi,” representing the same phenomenon.
